just a pixel jaccard calculation helper

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
#print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

In [ ]:
from shapely.geometry import MultiPolygon, Polygon
import shapely.wkt
import shapely.affinity
import cv2, os, sys, csv
csv.field_size_limit(sys.maxsize)


In [ ]:


GS = pd.read_csv(os.path.join('..','input', 'grid_sizes.csv'), names=['ImageId', 'Xmax', 'Ymin'], skiprows=1)


In [ ]:

smooth = 1e-12

# helper function to compute jaccard based on masks not polygons
def compute_jaccard_from_masks( pred, target):
    # expect to have pred shape as (n_cls, width, height) and target as well
    assert(len(pred.shape) == 3)
    assert(pred.shape == target.shape)
    
    # expect to have pred and target type set to bool
    assert(pred.dtype == np.bool)
    assert(target.dtype == np.bool)
    
    matching_pixels = np.sum(pred * target, axis=(1,2)).astype(np.float64)
    false_positive = np.sum( pred * (1. - target), axis = (1,2)).astype(np.float64)
    false_negative = np.sum( (1 - pred) * target, axis = (1,2)).astype(np.float64)
    
    
    jaccard = matching_pixels / ((matching_pixels + false_positive  + false_negative ) + smooth)
    
    return jaccard
    

In [ ]:
# simple test of the function
fake_target = np.ones((10,2,3), dtype=np.bool)
fake_pred = np.zeros((10,2,3), dtype=np.bool)
fake_pred[0,1,:] = True
fake_pred[8] = True

jaccard_by_class = compute_jaccard_from_masks(fake_pred, fake_target)
for cls, score in enumerate(jaccard_by_class):
    print('score for class {} is {}'.format(cls+1, score))


In [ ]:
def get_scalers(im_size, x_max, y_min):
    # __author__ = Konstantin Lopuhin
    # https://www.kaggle.com/lopuhin/dstl-satellite-imagery-feature-detection/full-pipeline-demo-poly-pixels-ml-poly
    h, w = im_size  # they are flipped so that mask_for_polygons works correctly
    h, w = float(h), float(w)
    w_ = 1.0 * w * (w / (w + 1))
    h_ = 1.0 * h * (h / (h + 1))
    return w_ / x_max, h_ / y_min



def mask_for_polygons(polygons, im_size):
    # based from 
    # __author__ = Konstantin Lopuhin
    # https://www.kaggle.com/lopuhin/dstl-satellite-imagery-feature-detection/full-pipeline-demo-poly-pixels-ml-poly
    img_mask = np.zeros(im_size, np.uint8)
    if not polygons:
        return img_mask
    int_coords = lambda x: np.array(x).round().astype(np.int32)
    exteriors = [int_coords(poly.exterior.coords) for poly in polygons]
    interiors = [int_coords(pi.coords) for poly in polygons
                 for pi in poly.interiors]
    cv2.fillPoly(img_mask, exteriors, 1)
    cv2.fillPoly(img_mask, interiors, 0)
    return (img_mask > 0).astype(np.bool)




def scale_polygons(mpolygon, x_scaler, y_scaler):
    return shapely.affinity.scale(mpolygon, xfact=x_scaler, yfact=y_scaler,
                                                      origin=(0, 0, 0))





In [ ]:

def get_scalers_for_image(imageid, mask_size):
    x_max = GS.loc[GS['ImageId'] == imageid, 'Xmax'].as_matrix()[0]
    y_min = GS.loc[GS['ImageId'] == imageid, 'Ymin'].as_matrix()[0]
    return get_scalers(mask_size, x_max, y_min)


def scale_wkt_to_mask(mpolygon, x_scaler, y_scaler):
    return shapely.affinity.scale(mpolygon, xfact=x_scaler, yfact=y_scaler,
                                                      origin=(0, 0, 0))




In [ ]:
if not os.path.isfile('fake_pred.csv'):
    with open('fake_pred.csv','w') as fp:
        writer = csv.writer(fp)
        writer.writerow(['ImageId','ClassType','MultipolygonWKT'])
        writer.writerow(['6010_1_2','1','GEOMETRYCOLLECTION EMPTY'])
        writer.writerow(['6010_1_2','2','GEOMETRYCOLLECTION EMPTY'])
        writer.writerow(['6010_1_2','3','GEOMETRYCOLLECTION EMPTY'])
        writer.writerow(['6010_1_2','4','GEOMETRYCOLLECTION EMPTY'])
        writer.writerow(['6010_1_2','5',"MULTIPOLYGON (((0.0091608969965358 -0.0020822525025185, 0.0091365903712502 -0.0021146611017795, 0.0091527947881073 -0.0021335661180151, 0.0091608969965358 -0.0020822525025185)), ((0.0091581962603930 -0.0019715231217102, 0.0090717727038219 -0.0019796252715254, 0.0090582690231076 -0.0020147345873915, 0.0090798749122504 -0.0020336396036271, 0.0091581962603930 -0.0019958295711559, 0.0091581962603930 -0.0019715231217102)), ((0.0090123565086792 -0.0016717435785460, 0.0089799476749651 -0.0017635679431188, 0.0090447653423934 -0.0017797722427493, 0.0090609697592505 -0.0017149550442273, 0.0090960793291075 -0.0016987507445968, 0.0090825756483932 -0.0016825464449663, 0.0090285609255363 -0.0016987507445968, 0.0090123565086792 -0.0016717435785460)), ((0.0090852763845361 -0.0018769980405323, 0.0090015535641078 -0.0018932023401628, 0.0090582690231076 -0.0019067059231882, 0.0090852763845361 -0.0018769980405323)), ((0.0090636704953933 -0.0021902811667218, 0.0090123565086792 -0.0021848797335117, 0.0090015535641078 -0.0022226897659828, 0.0090447653423934 -0.0022226897659828, 0.0090636704953933 -0.0021902811667218)), ((0.0090447653423934 -0.0004024067741573, 0.0089664439942508 -0.0004429175232335, 0.0089718454665365 -0.0005023332885453, 0.0090015535641078 -0.0005374426044114, 0.0090582690231076 -0.0004861289889148, 0.0090447653423934 -0.0004024067741573)), ((0.0089448381051081 -0.0068463165938839, 0.0089016263268225 -0.0068814259097500, 0.0089016263268225 -0.0069570459746923, 0.0089637432581080 -0.0069975567237685, 0.0090231594532506 -0.0069705495577177, 0.0090528675508219 -0.0068949294927754, 0.0090177579809649 -0.0068571194603043, 0.0089448381051081 -0.0068463165938839)), ((0.0089961520918222 -0.0003753996081065, 0.0089745462026794 -0.0003645967416861, 0.0089124292713939 -0.0003943046243420, 0.0088827211738226 -0.0003753996081065, 0.0088773197015369 -0.0004483189564437, 0.0089961520918222 -0.0003753996081065)), ((0.0089826484111079 -0.0009722579778297, 0.0089610425219651 -0.0009749586944348, 0.0089610425219651 -0.0009992651438805, 0.0089826484111079 -0.0009722579778297)), ((0.0089367358966795 -0.0018851001903475, 0.0089583417858223 -0.0019040052065831, 0.0089799476749651 -0.0018932023401628, 0.0089637432581080 -0.0018742973239272, 0.0089367358966795 -0.0018851001903475)), ((0.0088449108678227 -0.0019553188220797, 0.0088746189653941 -0.0019958295711559, 0.0089232322159653 -0.0019877274213407, 0.0089394366328224 -0.0020066324375762, 0.0089718454665365 -0.0019607202552898, 0.0089097285352511 -0.0019283116560288, 0.0088665167569655 -0.0019283116560288, 0.0088449108678227 -0.0019553188220797)), ((0.0089475388412509 -0.0081291569812981, 0.0089259329521081 -0.0081264562646930, 0.0089097285352511 -0.0081615655805591, 0.0089475388412509 -0.0081291569812981)), ((0.0089232322159653 -0.0020201360206016, 0.0089313344243938 -0.0020606467696779, 0.0089421373689652 -0.0020255374538118, 0.0089232322159653 -0.0020201360206016)), ((0.0088233049786800 -0.0013611611689615, 0.0088152027702514 -0.0013908690516174, 0.0088449108678227 -0.0013827669018022, 0.0088773197015369 -0.0014502848169293, 0.0089259329521081 -0.0014151755010632, 0.0088719182292512 -0.0013800661851971, 0.0088989255906797 -0.0013449568693310, 0.0088233049786800 -0.0013611611689615)), ((0.0089259329521081 -0.0016258313962596, 0.0088800204376797 -0.0016312328294698, 0.0088719182292512 -0.0016744442951511, 0.0089097285352511 -0.0016690428619409, 0.0089259329521081 -0.0016258313962596)), ((0.0089205314798224 -0.0009155429291229, 0.0088800204376797 -0.0009479515283839, 0.0088881226461083 -0.0009722579778297, 0.0089124292713939 -0.0009587543948043, 0.0089205314798224 -0.0009155429291229)), ((0.0088287064509657 -0.0018418887246662, 0.0087962976172515 -0.0018472901578764, 0.0087611880473945 -0.0018932023401628, 0.0088098012979657 -0.0018823994737424, 0.0088287064509657 -0.0018418887246662)), ((0.0088287064509657 -0.0018418887246662, 0.0088692174931084 -0.0018310858582459, 0.0088854219099654 -0.0018067794088001, 0.0088746189653941 -0.0017554657933036, 0.0088449108678227 -0.0017581665099087, 0.0088287064509657 -0.0018418887246662)), ((0.0087206770052518 -0.0064898220020131, 0.0086342534486807 -0.0065357341842995, 0.0086315527125379 -0.0065816463665859, 0.0086612608101092 -0.0066032520994265, 0.0086369541848235 -0.0066410621318977, 0.0086585600739663 -0.0066734707311587, 0.0087476843666803 -0.0067193829134451, 0.0088260057148228 -0.0066599671481332, 0.0088449108678227 -0.0065843470831909, 0.0087827939365373 -0.0064952234352232, 0.0087206770052518 -0.0064898220020131), (0.0087017718522519 -0.0065546392005350, 0.0087233777413946 -0.0065654420669554, 0.0087098740606804 -0.0065843470831909, 0.0087017718522519 -0.0065546392005350)), ((0.0087773924642516 -0.0005968583697232, 0.0087827939365373 -0.0006373691187995, 0.0088260057148228 -0.0006265662523791, 0.0088233049786800 -0.0005995590863283, 0.0087773924642516 -0.0005968583697232)), ((0.0086882681715376 -0.0017797722427493, 0.0087017718522519 -0.0018256844250357, 0.0087449836305374 -0.0017959765423798, 0.0088016990895372 -0.0017959765423798, 0.0087260784775375 -0.0017662686597239, 0.0086882681715376 -0.0017797722427493)), ((0.0087422828943946 -0.0021011575187541, 0.0087233777413946 -0.0021092596685694, 0.0087530858389660 -0.0021308654014100, 0.0087179762691089 -0.0021875804501168, 0.0087287792136803 -0.0022037847497473, 0.0087854946726801 -0.0021389675512253, 0.0087422828943946 -0.0021011575187541)), ((0.0087368814221089 -0.0006913834509011, 0.0087287792136803 -0.0007129891837418, 0.0087530858389660 -0.0007318941999773, 0.0087557865751088 -0.0007021863173214, 0.0087368814221089 -0.0006913834509011)), ((0.0086909689076805 -0.0016150285298393, 0.0086666622823949 -0.0016717435785460, 0.0087179762691089 -0.0016798457283613, 0.0087395821582517 -0.0016528385623104, 0.0086909689076805 -0.0016150285298393)), ((0.0087341806859660 -0.0020066324375762, 0.0086990711161090 -0.0020147345873915, 0.0086612608101092 -0.0020606467696779, 0.0087233777413946 -0.0020471431866525, 0.0087341806859660 -0.0020066324375762)), ((0.0087314799498232 -0.0009398493785687, 0.0086747644908234 -0.0009506522449890, 0.0086801659631091 -0.0009803601276449, 0.0087260784775375 -0.0009749586944348, 0.0087314799498232 -0.0009398493785687)), ((0.0086342534486807 -0.0015907220803935, 0.0086666622823949 -0.0015826199305783, 0.0086531586016806 -0.0015718170641580, 0.0086342534486807 -0.0015907220803935)), ((0.0086612608101092 -0.0008507257306010, 0.0086234505041093 -0.0008426235807857, 0.0086018446149665 -0.0008966379128874, 0.0085532313643953 -0.0009101414959128, 0.0085559321005382 -0.0009803601276449, 0.0086315527125379 -0.0009749586944348, 0.0086180490318236 -0.0009020393460975, 0.0086612608101092 -0.0008507257306010)), ((0.0086585600739663 -0.0007345949165824, 0.0085775379896809 -0.0007913099652891, 0.0085802387258238 -0.0008318207143654, 0.0086018446149665 -0.0008399228641806, 0.0086207497679665 -0.0008075142649196, 0.0086450563932521 -0.0008075142649196, 0.0086585600739663 -0.0007345949165824)), ((0.0086396549209664 -0.0013395554361209, 0.0085343262113954 -0.0013503583025412, 0.0085343262113954 -0.0013989712014327, 0.0086045453511094 -0.0013881683350124, 0.0086423556571092 -0.0013611611689615, 0.0086396549209664 -0.0013395554361209)), ((0.0086207497679665 -0.0006589748516401, 0.0086018446149665 -0.0006940841675062, 0.0086288519763950 -0.0006805805844808, 0.0086207497679665 -0.0006589748516401)), ((0.0086126475595379 -0.0057930371179018, 0.0085262240029668 -0.0058119421341374, 0.0084587055993957 -0.0059064672153153, 0.0084587055993957 -0.0059469779643915, 0.0085019173776812 -0.0059847879968626, 0.0084722092801099 -0.0060415030455694, 0.0084803114885384 -0.0060847145112507, 0.0085154210583955 -0.0061117216773015, 0.0085694357812524 -0.0060982180942761, 0.0086099468233951 -0.0060334008957541, 0.0085586328366810 -0.0059523793976017, 0.0086207497679665 -0.0058362485835831, 0.0086126475595379 -0.0057930371179018)), ((0.0085964431426809 -0.0055688776396799, 0.0085154210583955 -0.0055661769230749, 0.0084262967656815 -0.0056174905385714, 0.0084479026548243 -0.0057066141865392, 0.0085586328366810 -0.0056661034374629, 0.0085964431426809 -0.0056093883887562, 0.0085964431426809 -0.0055688776396799)), ((0.0085640343089667 -0.0065789456499808, 0.0085667350451095 -0.0066059528160316, 0.0085856401981095 -0.0066059528160316, 0.0085640343089667 -0.0065789456499808)), ((0.0085559321005382 -0.0041023885231202, 0.0085640343089667 -0.0041239942559609, 0.0085856401981095 -0.0041131913895405, 0.0085775379896809 -0.0040942863733049, 0.0085559321005382 -0.0041023885231202)), ((0.0085694357812524 -0.0021929818833269, 0.0084506033909671 -0.0021713761504863, 0.0084262967656815 -0.0022307919157981, 0.0084506033909671 -0.0022442954988235, 0.0084803114885384 -0.0022118868995625, 0.0085694357812524 -0.0021929818833269)), ((0.0085289247391097 -0.0006697777180604, 0.0085127203222526 -0.0006751791512706, 0.0085073188499669 -0.0007318941999773, 0.0085478298921096 -0.0007183906169519, 0.0085289247391097 -0.0006697777180604)), ((0.0085451291559668 -0.0023496234464217, 0.0085208225306811 -0.0023253169969760, 0.0084911144331098 -0.0023604263128420, 0.0085343262113954 -0.0023685284626573, 0.0085451291559668 -0.0023496234464217)), ((0.0084776107523956 -0.0011505052737651, 0.0084938151692527 -0.0011694102900007, 0.0085181217945383 -0.0011586074235804, 0.0084776107523956 -0.0011505052737651)), ((0.0084235960295387 -0.0016069263800240, 0.0084343989741101 -0.0016204299630495, 0.0084614063355385 -0.0016015249468139, 0.0084235960295387 -0.0016069263800240)), ((0.0082939606946820 -0.0012504317881532, 0.0082777562778249 -0.0012720375209938, 0.0083074643753963 -0.0013152489866751, 0.0083371724729676 -0.0012990446870446, 0.0083398732091104 -0.0012720375209938, 0.0082939606946820 -0.0012504317881532)), ((0.0083263695283962 -0.0044426788153606, 0.0082993621669677 -0.0044426788153606, 0.0082993621669677 -0.0044669852648063, 0.0083263695283962 -0.0044426788153606)), ((0.0081913327212539 -0.0003591953084760, 0.0082048364019681 -0.0004078082073674, 0.0082183400826823 -0.0003943046243420, 0.0081913327212539 -0.0003591953084760)), ((0.0081535224152540 -0.0044939924308571, 0.0081589238875397 -0.0045128974470927, 0.0081778290405396 -0.0045074960138826, 0.0081535224152540 -0.0044939924308571)), ((0.0080670988586829 -0.0003429910088455, 0.0080400914972544 -0.0003781003247115, 0.0080562959141115 -0.0003808010413166, 0.0080670988586829 -0.0003429910088455)), ((0.0080535951779687 -0.0002484659276676, 0.0080076826635403 -0.0002295609114320, 0.0079590694129690 -0.0003024802597692, 0.0080265878165402 -0.0002943781099540, 0.0080535951779687 -0.0002484659276676)), ((0.0079995804551117 -0.0045993203784554, 0.0079914782466832 -0.0046776411600027, 0.0080265878165402 -0.0046695390101875, 0.0080238870803974 -0.0046209261112960, 0.0079995804551117 -0.0045993203784554)), ((0.0080238870803974 -0.0045399046131435, 0.0080022811912546 -0.0045372038965385, 0.0079833760382547 -0.0045669117791944, 0.0080076826635403 -0.0045723132124045, 0.0080238870803974 -0.0045399046131435)), ((0.0080022811912546 -0.0003997060575522, 0.0079752738298261 -0.0003916039077370, 0.0079563686768262 -0.0004159103571827, 0.0079806753021118 -0.0004321146568132, 0.0080022811912546 -0.0003997060575522)), ((0.0079995804551117 -0.0011424031239499, 0.0079752738298261 -0.0011586074235804, 0.0079806753021118 -0.0011910160228413, 0.0079995804551117 -0.0011424031239499)), ((0.0079644708852547 -0.0044183723659148, 0.0079509672045405 -0.0044507809651758, 0.0079860767743975 -0.0044642845482012, 0.0079995804551117 -0.0044237737991250, 0.0079644708852547 -0.0044183723659148)), ((0.0079752738298261 -0.0047289547754993, 0.0079212591069692 -0.0047073490426587, 0.0079050546901121 -0.0047316554921044, 0.0078645436479694 -0.0047262540588942, 0.0078321348142553 -0.0047559619415501, 0.0078024267166839 -0.0047370569253146, 0.0078051274528268 -0.0047667648079705, 0.0077403097853985 -0.0048477863061229, 0.0077484119938270 -0.0048828956219890, 0.0078186311335410 -0.0048639906057534, 0.0078780473286836 -0.0048828956219890, 0.0079482664683977 -0.0048585891725433, 0.0079833760382547 -0.0048153777068620, 0.0079752738298261 -0.0047289547754993)), ((0.0079698723575404 -0.0017284586272527, 0.0079401642599691 -0.0017743708095391, 0.0079644708852547 -0.0017662686597239, 0.0079698723575404 -0.0017284586272527)), ((0.0079536679406833 -0.0041915121710879, 0.0079077554262550 -0.0042131179039286, 0.0079077554262550 -0.0042455265031896, 0.0079644708852547 -0.0042347236367692, 0.0079536679406833 -0.0041915121710879)), ((0.0079455657322548 -0.0007994121151044, 0.0079050546901121 -0.0008264192811552, 0.0079131568985407 -0.0008561271638111, 0.0079590694129690 -0.0008399228641806, 0.0079455657322548 -0.0007994121151044)), ((0.0079536679406833 -0.0043454530175776, 0.0079293613153977 -0.0043535551673929, 0.0079293613153977 -0.0043886644832589, 0.0079482664683977 -0.0043832630500488, 0.0079536679406833 -0.0043454530175776)), ((0.0079428649961120 -0.0038458204456374, 0.0079185583708263 -0.0038458204456374, 0.0079023539539693 -0.0038863311947136, 0.0079428649961120 -0.0038458204456374)), ((0.0078996532178264 -0.0037350910648290, 0.0078591421756837 -0.0037593975142747, 0.0078213318696839 -0.0037539960810646, 0.0077889230359697 -0.0037783025305103, 0.0077889230359697 -0.0038080104131662, 0.0078996532178264 -0.0037350910648290)), ((0.0078969524816836 -0.0008993386294924, 0.0078510399672552 -0.0008912364796772, 0.0078105289251125 -0.0009209443623331, 0.0078726458563980 -0.0009398493785687, 0.0078969524816836 -0.0008993386294924)), ((0.0078942517455407 -0.0012720375209938, 0.0078672443841123 -0.0012720375209938, 0.0078618429118266 -0.0013044461202548, 0.0078942517455407 -0.0012720375209938)), ((0.0078861495371122 -0.0036675731497019, 0.0078186311335410 -0.0036567702832816, 0.0077781200913983 -0.0036729745829121, 0.0077592149383984 -0.0037593975142747, 0.0077970252443982 -0.0037404924980392, 0.0077943245082554 -0.0037080838987782, 0.0078699451202551 -0.0036999817489629, 0.0078861495371122 -0.0036675731497019)), ((0.0078753465925408 -0.0016825464449663, 0.0078429377588266 -0.0017311593438578, 0.0078834488009693 -0.0017176557608324, 0.0078753465925408 -0.0016825464449663)), ((0.0078537407033980 -0.0043454530175776, 0.0078078281889696 -0.0043481537341827, 0.0078105289251125 -0.0043994673496793, 0.0078591421756837 -0.0043670587504183, 0.0078537407033980 -0.0043454530175776)), ((0.0077349083131128 -0.0041401985555913, 0.0077484119938270 -0.0041564028552218, 0.0078051274528268 -0.0041347971223812, 0.0078456384949695 -0.0041483007054066, 0.0078537407033980 -0.0041239942559609, 0.0078159303973982 -0.0041266949725659, 0.0077754193552555 -0.0040996878065151, 0.0077349083131128 -0.0041401985555913)), ((0.0078267333419696 -0.0042050157541133, 0.0077943245082554 -0.0041996143209032, 0.0077511127299699 -0.0042482272197946, 0.0077889230359697 -0.0042428257865845, 0.0078267333419696 -0.0042050157541133)), ((0.0078240326058267 -0.0012909425372294, 0.0077997259805411 -0.0013044461202548, 0.0077889230359697 -0.0013476575859361, 0.0078105289251125 -0.0013395554361209, 0.0078240326058267 -0.0012909425372294)), ((0.0077862222998269 -0.0038323168626120, 0.0077376090492556 -0.0038593240286628, 0.0077484119938270 -0.0038809297615034, 0.0078051274528268 -0.0038620247452679, 0.0077862222998269 -0.0038323168626120)), ((0.0077484119938270 -0.0007913099652891, 0.0077997259805411 -0.0008210178479451, 0.0077835215636840 -0.0007832078154739, 0.0077484119938270 -0.0007913099652891)), ((0.0077214046323985 -0.0018742973239272, 0.0077538134661127 -0.0018959030567679, 0.0077808208275412 -0.0018688958907170, 0.0077268061046842 -0.0018580930242967, 0.0077214046323985 -0.0018742973239272)), ((0.0077781200913983 -0.0035811502183393, 0.0077349083131128 -0.0035919530847596, 0.0077403097853985 -0.0036270624006257, 0.0077592149383984 -0.0036270624006257, 0.0077781200913983 -0.0035811502183393)), ((0.0077754193552555 -0.0009479515283839, 0.0077268061046842 -0.0009506522449890, 0.0077295068408271 -0.0009992651438805, 0.0077754193552555 -0.0009479515283839)), ((0.0077700178829698 -0.0010181701601161, 0.0077133024239700 -0.0010586809091923, 0.0077160031601128 -0.0010883887918482, 0.0077700178829698 -0.0010802866420330, 0.0077700178829698 -0.0010181701601161)), ((0.0076862950625415 -0.0000027007166051, 0.0076970980071129 -0.0001107293808084, 0.0077214046323985 -0.0001215322472287, 0.0077646164106841 -0.0000945250811779, 0.0077700178829698 -0.0000621164819169, 0.0077295068408271 -0.0000540143321016, 0.0077349083131128 -0.0000243064494457, 0.0076862950625415 -0.0000027007166051)), ((0.0076457840203988 -0.0046236268279011, 0.0076403825481131 -0.0046641375769773, 0.0077241053685414 -0.0046425318441367, 0.0077403097853985 -0.0046722397267926, 0.0077646164106841 -0.0046479332773468, 0.0077376090492556 -0.0046020210950604, 0.0076835943263987 -0.0045993203784554, 0.0076457840203988 -0.0046236268279011)), ((0.0077592149383984 -0.0017122543276223, 0.0077052002155415 -0.0017149550442273, 0.0077241053685414 -0.0017608672265137, 0.0077592149383984 -0.0017122543276223)), ((0.0077538134661127 -0.0012693368043887, 0.0077133024239700 -0.0012720375209938, 0.0076997987432558 -0.0013530590191463, 0.0077457112576842 -0.0013233511364904, 0.0077538134661127 -0.0012693368043887)), ((0.0076565869649702 -0.0040024620087321, 0.0076457840203988 -0.0040321698913881, 0.0076673899095416 -0.0040429727578084, 0.0076646891733988 -0.0040888849400948, 0.0076349810758274 -0.0041158921061456, 0.0076646891733988 -0.0041618042884320, 0.0076457840203988 -0.0041888114544828, 0.0076754921179701 -0.0042266214869540, 0.0077106016878272 -0.0042293222035591, 0.0077214046323985 -0.0041888114544828, 0.0076943972709701 -0.0041645050050371, 0.0077214046323985 -0.0041401985555913, 0.0076781928541130 -0.0040969870899100, 0.0076943972709701 -0.0040186663083626, 0.0076565869649702 -0.0040024620087321)), ((0.0076997987432558 -0.0022767040980845, 0.0076781928541130 -0.0022740033814794, 0.0076646891733988 -0.0022983098309251, 0.0076997987432558 -0.0022767040980845)), ((0.0076835943263987 -0.0009695572612246, 0.0076349810758274 -0.0009857615608551, 0.0076592877011131 -0.0010613816257974, 0.0076943972709701 -0.0009911629940652, 0.0076835943263987 -0.0009695572612246)), ((0.0076376818119703 -0.0038134118463764, 0.0076457840203988 -0.0038323168626120, 0.0076889957986844 -0.0038188132795865, 0.0076376818119703 -0.0038134118463764)), ((0.0076430832842560 -0.0044345766655453, 0.0076214773951132 -0.0044723866980165, 0.0076484847565417 -0.0044588831149911, 0.0076430832842560 -0.0044345766655453)), ((0.0076106744505418 -0.0019283116560288, 0.0076187766589704 -0.0019580195386848, 0.0076457840203988 -0.0019499173888695, 0.0076430832842560 -0.0019310123726339, 0.0076106744505418 -0.0019283116560288)), ((0.0075080464771137 -0.0046884440264231, 0.0075161486855422 -0.0047235533422892, 0.0075701634083991 -0.0047073490426587, 0.0075593604638277 -0.0047532612249451, 0.0076349810758274 -0.0047235533422892, 0.0076241781312561 -0.0047046483260536, 0.0075863678252562 -0.0047100497592637, 0.0075647619361134 -0.0046614368603723, 0.0075080464771137 -0.0046884440264231)), ((0.0076295796035418 -0.0037674996640900, 0.0075998715059704 -0.0038053096965611, 0.0076295796035418 -0.0038026089799560, 0.0076295796035418 -0.0037674996640900)), ((0.0075998715059704 -0.0005212383047809, 0.0075512582553992 -0.0005320411712012, 0.0075431560469707 -0.0005563476206470, 0.0075917692975419 -0.0005536469040419, 0.0075998715059704 -0.0005212383047809)), ((0.0075512582553992 -0.0013800661851971, 0.0075215501578279 -0.0014016719180378, 0.0075161486855422 -0.0014745912663750, 0.0075323531023993 -0.0014961969992157, 0.0075971707698276 -0.0014529855335343, 0.0075512582553992 -0.0013800661851971)), ((0.0075782656168277 -0.0022902076811099, 0.0075296523662564 -0.0022956091143201, 0.0075161486855422 -0.0023280177135810, 0.0075539589915420 -0.0023280177135810, 0.0075782656168277 -0.0022902076811099)), ((0.0075728641445420 -0.0043130444183166, 0.0075512582553992 -0.0043427523009725, 0.0075728641445420 -0.0043373508677624, 0.0075728641445420 -0.0043130444183166)), ((0.0075458567831135 -0.0041861107378777, 0.0075323531023993 -0.0042401250699794, 0.0075620611999706 -0.0042482272197946, 0.0075728641445420 -0.0042212200537438, 0.0075458567831135 -0.0041861107378777)), ((0.0075512582553992 -0.0035028294367919, 0.0075107472132565 -0.0035217344530275, 0.0075350538385421 -0.0035271358862376, 0.0075512582553992 -0.0035028294367919)), ((0.0075242508939707 -0.0010829873586381, 0.0075161486855422 -0.0010964909416635, 0.0075458567831135 -0.0011099945246889, 0.0075242508939707 -0.0010829873586381)), ((0.0075080464771137 -0.0008588278804162, 0.0074918420602566 -0.0008912364796772, 0.0075215501578279 -0.0009290465121484, 0.0075296523662564 -0.0008777328966518, 0.0075080464771137 -0.0008588278804162)), ((0.0074081192398283 -0.0044507809651758, 0.0074729369072566 -0.0044750874146216, 0.0075080464771137 -0.0044372773821504, 0.0074459295458282 -0.0044048687828894, 0.0074081192398283 -0.0044507809651758)), ((0.0075053457409708 -0.0041483007054066, 0.0074756376433995 -0.0041726071548523, 0.0075053457409708 -0.0041672057216422, 0.0075053457409708 -0.0041483007054066)), ((0.0074945427963994 -0.0046641375769773, 0.0074621339626853 -0.0046614368603723, 0.0074405280735425 -0.0047019476094485, 0.0074945427963994 -0.0046641375769773)), ((0.0074621339626853 -0.0041861107378777, 0.0074108199759712 -0.0042131179039286, 0.0074648346988281 -0.0042131179039286, 0.0074621339626853 -0.0041861107378777)), ((0.0074486302819710 -0.0008858350464670, 0.0074000170313998 -0.0009074407793077, 0.0074243236566854 -0.0009533529615941, 0.0074594332265424 -0.0009371486619636, 0.0074486302819710 -0.0008858350464670)), ((0.0074567324903996 -0.0043454530175776, 0.0074162214482569 -0.0043508544507878, 0.0074081192398283 -0.0043724601836284, 0.0074459295458282 -0.0043832630500488, 0.0074567324903996 -0.0043454530175776)), ((0.0074540317542567 -0.0018121808420103, 0.0074378273373996 -0.0018364872914560, 0.0074081192398283 -0.0018364872914560, 0.0074162214482569 -0.0018607937409018, 0.0074459295458282 -0.0018553923076916, 0.0074540317542567 -0.0018121808420103)), ((0.0074405280735425 -0.0011640088567905, 0.0073865133506856 -0.0011829138730261, 0.0073892140868284 -0.0012099210390769, 0.0074135207121140 -0.0012072203224718, 0.0074405280735425 -0.0011640088567905)), ((0.0074189221843997 -0.0034407129548750, 0.0073892140868284 -0.0034569172545055, 0.0074054185036855 -0.0034839244205563, 0.0074351266012568 -0.0034596179711106, 0.0074189221843997 -0.0034407129548750)), ((0.0073351993639715 -0.0038323168626120, 0.0074351266012568 -0.0038215139961916, 0.0073784111422570 -0.0038107111297713, 0.0073351993639715 -0.0038323168626120)), ((0.0074297251289711 -0.0086557967192892, 0.0073973162952569 -0.0086665995857095, 0.0073838126145427 -0.0087098110513908, 0.0074297251289711 -0.0086909060351552, 0.0074297251289711 -0.0086557967192892)), ((0.0074189221843997 -0.0020093331541813, 0.0073676081976856 -0.0020282381704169, 0.0073703089338285 -0.0020660482028881, 0.0074189221843997 -0.0020336396036271, 0.0074189221843997 -0.0020093331541813)), ((0.0074108199759712 -0.0048639906057534, 0.0073324986278286 -0.0048774941887788, 0.0073784111422570 -0.0049018006382246, 0.0074108199759712 -0.0048639906057534)), ((0.0074054185036855 -0.0017419622102782, 0.0073324986278286 -0.0017716700929341, 0.0073351993639715 -0.0018067794088001, 0.0073595059892571 -0.0018121808420103, 0.0073730096699713 -0.0017770715261442, 0.0074081192398283 -0.0017716700929341, 0.0074054185036855 -0.0017419622102782)), ((0.0073973162952569 -0.0045912182286401, 0.0073811118783999 -0.0045912182286401, 0.0073838126145427 -0.0046209261112960, 0.0073973162952569 -0.0045912182286401)), ((0.0073919148229712 -0.0045291017467232, 0.0073595059892571 -0.0045318024633283, 0.0073514037808285 -0.0045831160788249, 0.0073811118783999 -0.0045804153622198, 0.0073919148229712 -0.0045291017467232)), ((0.0073487030446857 -0.0036648724330968, 0.0073622067253999 -0.0036810767327273, 0.0073838126145427 -0.0036702738663070, 0.0073487030446857 -0.0036648724330968)), ((0.0073676081976856 -0.0047046483260536, 0.0073270971555429 -0.0047397576419196, 0.0073243964194001 -0.0047775676743908, 0.0073811118783999 -0.0047451590751298, 0.0073676081976856 -0.0047046483260536)), ((0.0073757104061142 -0.0018959030567679, 0.0073433015724000 -0.0019040052065831, 0.0073379001001143 -0.0019337130892390, 0.0073703089338285 -0.0019337130892390, 0.0073757104061142 -0.0018959030567679)), ((0.0073595059892571 -0.0037485946478544, 0.0073108927386859 -0.0037539960810646, 0.0072865861134002 -0.0038080104131662, 0.0073162942109715 -0.0038296161460069, 0.0073162942109715 -0.0037999082633510, 0.0073730096699713 -0.0038026089799560, 0.0073595059892571 -0.0037485946478544)), ((0.0072649802242575 -0.0049720192699567, 0.0073135934748287 -0.0050584422013194, 0.0073324986278286 -0.0050044278692177, 0.0073703089338285 -0.0049963257194025, 0.0073541045169714 -0.0049720192699567, 0.0072649802242575 -0.0049720192699567)), ((0.0072406735989718 -0.0010586809091923, 0.0072514765435432 -0.0010937902250584, 0.0073541045169714 -0.0011180966745041, 0.0073243964194001 -0.0010370751763516, 0.0072784839049717 -0.0010343744597466, 0.0072406735989718 -0.0010586809091923)), ((0.0073487030446857 -0.0045912182286401, 0.0072973890579716 -0.0046263275445062, 0.0073243964194001 -0.0046533347105570, 0.0073487030446857 -0.0046344296943214, 0.0073487030446857 -0.0045912182286401)), ((0.0073351993639715 -0.0022550983652438, 0.0072838853772574 -0.0022740033814794, 0.0073108927386859 -0.0023091126973455, 0.0073460023085429 -0.0022875069645048, 0.0073351993639715 -0.0022550983652438)), ((0.0073406008362572 -0.0044237737991250, 0.0073054912664002 -0.0044183723659148, 0.0072325713905433 -0.0044696859814114, 0.0073135934748287 -0.0044588831149911, 0.0073406008362572 -0.0044237737991250)), ((0.0073216956832572 -0.0012180231888922, 0.0073081920025430 -0.0012369282051277, 0.0073324986278286 -0.0012477310715481, 0.0073406008362572 -0.0012288260553125, 0.0073216956832572 -0.0012180231888922)), ((0.0072811846411145 -0.0013665626021717, 0.0072973890579716 -0.0013935697682225, 0.0073406008362572 -0.0013854676184073, 0.0073000897941145 -0.0013557597357514, 0.0072811846411145 -0.0013665626021717)), ((0.0073379001001143 -0.0043292487179471, 0.0072973890579716 -0.0043400515843674, 0.0073081920025430 -0.0043724601836284, 0.0073379001001143 -0.0043292487179471)), ((0.0071974618206863 -0.0038863311947136, 0.0072379728628290 -0.0038890319113187, 0.0072298706544005 -0.0039403455268152, 0.0073108927386859 -0.0038782290448984, 0.0072217684459719 -0.0038620247452679, 0.0071974618206863 -0.0038863311947136)), ((0.0073054912664002 -0.0020228367372067, 0.0072514765435432 -0.0020417417534423, 0.0072541772796861 -0.0020606467696779, 0.0073054912664002 -0.0020471431866525, 0.0073054912664002 -0.0020228367372067)), ((0.0072892868495431 -0.0041591035718269, 0.0072757831688288 -0.0041969136042981, 0.0073027905302573 -0.0041780085880625, 0.0072892868495431 -0.0041591035718269)), ((0.0072676809604003 -0.0043724601836284, 0.0072190677098291 -0.0043805623334437, 0.0072163669736862 -0.0044075694994945, 0.0072649802242575 -0.0043994673496793, 0.0072676809604003 -0.0043724601836284)), ((0.0072217684459719 -0.0048585891725433, 0.0072001625568291 -0.0048666913223585, 0.0072109655014005 -0.0048936984884093, 0.0072217684459719 -0.0048585891725433)), ((0.0072109655014005 -0.0085828773709519, 0.0071974618206863 -0.0085720745045316, 0.0071272426809723 -0.0086017823871875, 0.0071164397364009 -0.0086233881200282, 0.0071866588761149 -0.0086206874034231, 0.0072109655014005 -0.0085828773709519)), ((0.0071974618206863 -0.0039619512596559, 0.0071461478339722 -0.0039754548426813, 0.0071488485701151 -0.0040051627253372, 0.0071245419448294 -0.0040240677415728, 0.0071353448894008 -0.0040726806404643, 0.0071893596122578 -0.0040510749076236, 0.0071812574038292 -0.0040024620087321, 0.0072109655014005 -0.0039835569924966, 0.0071974618206863 -0.0039619512596559)), ((0.0071893596122578 -0.0082452877953166, 0.0071542500424007 -0.0082695942447624, 0.0071785566676864 -0.0082695942447624, 0.0071893596122578 -0.0082452877953166)), ((0.0071812574038292 -0.0043373508677624, 0.0071515493062579 -0.0043886644832589, 0.0071866588761149 -0.0043643580338132, 0.0071812574038292 -0.0043373508677624)), ((0.0071596515146864 -0.0045264010301181, 0.0071812574038292 -0.0045588096293791, 0.0071812574038292 -0.0045264010301181, 0.0071596515146864 -0.0045264010301181)), ((0.0071731551954007 -0.0026034908072995, 0.0071434470978294 -0.0026304979733503, 0.0071623522508293 -0.0026331986899554, 0.0071731551954007 -0.0026034908072995)), ((0.0071704544592578 -0.0044723866980165, 0.0071083375279724 -0.0044507809651758, 0.0070462205966869 -0.0045507074795639, 0.0071650529869721 -0.0044939924308571, 0.0071704544592578 -0.0044723866980165)), ((0.0070543228051154 -0.0046938454596332, 0.0070624250135440 -0.0047100497592637, 0.0071326441532580 -0.0047019476094485, 0.0071569507785436 -0.0046614368603723, 0.0070678264858297 -0.0046614368603723, 0.0070543228051154 -0.0046938454596332)), ((0.0071542500424007 -0.0088772554809059, 0.0071218412086866 -0.0088610511812754, 0.0070867316388296 -0.0088745547643008, 0.0070624250135440 -0.0089177662299821, 0.0070759286942582 -0.0089636784122685, 0.0071380456255437 -0.0089231676631923, 0.0071542500424007 -0.0088772554809059)), ((0.0071326441532580 -0.0083236085768640, 0.0071488485701151 -0.0083479150263098, 0.0071542500424007 -0.0083290100100742, 0.0071326441532580 -0.0083236085768640)), ((0.0071272426809723 -0.0039079369275543, 0.0070489213328297 -0.0039160390773695, 0.0070408191244012 -0.0039511483932356, 0.0070030088184013 -0.0039376448102102, 0.0069922058738299 -0.0039727541260762, 0.0070192132352584 -0.0039835569924966, 0.0071056367918295 -0.0039511483932356, 0.0071272426809723 -0.0039079369275543)), ((0.0071110382641152 -0.0024306449445742, 0.0070948338472581 -0.0024711556936504, 0.0071245419448294 -0.0024549513940199, 0.0071110382641152 -0.0024306449445742)), ((0.0071110382641152 -0.0084559436905131, 0.0070624250135440 -0.0084910530063791, 0.0070705272219725 -0.0085126587392198, 0.0071191404725437 -0.0084775494233537, 0.0071110382641152 -0.0084559436905131)), ((0.0071164397364009 -0.0041807093046676, 0.0071002353195438 -0.0041753078714574, 0.0070894323749724 -0.0042023150375082, 0.0071164397364009 -0.0041807093046676)), ((0.0070921331111153 -0.0045615103459842, 0.0070381183882583 -0.0046182253946909, 0.0070624250135440 -0.0046344296943214, 0.0071029360556867 -0.0046020210950604, 0.0070921331111153 -0.0045615103459842)), ((0.0070813301665439 -0.0042428257865845, 0.0070489213328297 -0.0042860372522658, 0.0070894323749724 -0.0042698329526353, 0.0070813301665439 -0.0042428257865845)), ((0.0070840309026867 -0.0008318207143654, 0.0070624250135440 -0.0008264192811552, 0.0070543228051154 -0.0008507257306010, 0.0070840309026867 -0.0008318207143654)), ((0.0070786294304010 -0.0037972075467459, 0.0070435198605440 -0.0038242147127967, 0.0070192132352584 -0.0037918061135357, 0.0069841036654014 -0.0037999082633510, 0.0070084102906870 -0.0038782290448984, 0.0070786294304010 -0.0037972075467459)), ((0.0070705272219725 -0.0009371486619636, 0.0070435198605440 -0.0009614551114093, 0.0070489213328297 -0.0009884622774602, 0.0070705272219725 -0.0009803601276449, 0.0070705272219725 -0.0009371486619636)), ((0.0070003080822585 -0.0084046300750165, 0.0070354176521155 -0.0084397393908826, 0.0070651257496868 -0.0084289365244622, 0.0070003080822585 -0.0084046300750165)), ((0.0070111110268299 -0.0086801031687349, 0.0070057095545442 -0.0087125117679959, 0.0070408191244012 -0.0087152124846010, 0.0070354176521155 -0.0086801031687349, 0.0070111110268299 -0.0086801031687349)), ((0.0070084102906870 -0.0085180601724299, 0.0069516948316872 -0.0085126587392198, 0.0069408918871159 -0.0085477680550859, 0.0069138845256874 -0.0085450673384808, 0.0068976801088303 -0.0085855780875570, 0.0068706727474018 -0.0085882788041621, 0.0068922786365446 -0.0087017089015756, 0.0069165852618302 -0.0086990081849705, 0.0070003080822585 -0.0085909795207672, 0.0070300161798298 -0.0085801766543468, 0.0070084102906870 -0.0085180601724299)), ((0.0069814029292586 -0.0045209995969080, 0.0069841036654014 -0.0045426053297486, 0.0070246147075441 -0.0045318024633283, 0.0070057095545442 -0.0045101967304876, 0.0069814029292586 -0.0045209995969080)), ((0.0069895051376871 -0.0040132648751525, 0.0070138117629727 -0.0040267684581779, 0.0070084102906870 -0.0040078634419423, 0.0069895051376871 -0.0040132648751525)), ((0.0069814029292586 -0.0076673344418290, 0.0069543955678301 -0.0076754365916442, 0.0069651985124015 -0.0077753631060323, 0.0070084102906870 -0.0077267502071408, 0.0069814029292586 -0.0076673344418290)), ((0.0069949066099728 -0.0089204669465872, 0.0069597970401158 -0.0089258683797974, 0.0069408918871159 -0.0089906855783194, 0.0069651985124015 -0.0089933862949245, 0.0069949066099728 -0.0089204669465872)), ((0.0069841036654014 -0.0021308654014100, 0.0069354904148302 -0.0021497704176456, 0.0069435926232587 -0.0021794783003015, 0.0069895051376871 -0.0021605732840659, 0.0069841036654014 -0.0021308654014100)), ((0.0069787021931157 -0.0083344114432843, 0.0069435926232587 -0.0083317107266793, 0.0069354904148302 -0.0083614186093352, 0.0069651985124015 -0.0083614186093352, 0.0069787021931157 -0.0083344114432843)), ((0.0069787021931157 -0.0057417235024052, 0.0069246874702588 -0.0057525263688256, 0.0069192859979731 -0.0057714313850611, 0.0069787021931157 -0.0057417235024052)), ((0.0069516948316872 -0.0079049975030762, 0.0068976801088303 -0.0079536104019677, 0.0068922786365446 -0.0079995225842541, 0.0068166580245449 -0.0080589383495659, 0.0068760742196875 -0.0080643397827761, 0.0068922786365446 -0.0081075512484574, 0.0068814756919732 -0.0081615655805591, 0.0069057823172589 -0.0081534634307438, 0.0069273882064016 -0.0080481354831456, 0.0069624977762586 -0.0080103254506745, 0.0069273882064016 -0.0079725154182033, 0.0069624977762586 -0.0079428075355474, 0.0069516948316872 -0.0079049975030762)), ((0.0069543955678301 -0.0081993756130302, 0.0068895779004018 -0.0082101784794506, 0.0068895779004018 -0.0082911999776030, 0.0069381911509730 -0.0082722949613675, 0.0069543955678301 -0.0081993756130302)), ((0.0068031543438307 -0.0084586444071181, 0.0068328624414020 -0.0084775494233537, 0.0069138845256874 -0.0084775494233537, 0.0068625705389733 -0.0084289365244622, 0.0068031543438307 -0.0084586444071181)), ((0.0069057823172589 -0.0078455817377644, 0.0068733734835447 -0.0078428810211593, 0.0068625705389733 -0.0078617860373949, 0.0069057823172589 -0.0078455817377644)), ((0.0068490668582591 -0.0088637518978805, 0.0068706727474018 -0.0088799561975110, 0.0069057823172589 -0.0088421461650398, 0.0068544683305448 -0.0088313432986195, 0.0068490668582591 -0.0088637518978805)), ((0.0068895779004018 -0.0076403272757782, 0.0068193587606878 -0.0076241229761477, 0.0067437381486881 -0.0076484294255934, 0.0067437381486881 -0.0077105459075103, 0.0068058550799735 -0.0077186480573256, 0.0068895779004018 -0.0076403272757782)), ((0.0068571690666876 -0.0009236450789382, 0.0068220594968306 -0.0009452508117788, 0.0068409646498305 -0.0009830608442500, 0.0068814756919732 -0.0009506522449890, 0.0068571690666876 -0.0009236450789382)), ((0.0068517675944019 -0.0025656807748283, 0.0068679720112590 -0.0025845857910639, 0.0068652712751162 -0.0025602793416181, 0.0068517675944019 -0.0025656807748283)), ((0.0068544683305448 -0.0083074042772335, 0.0068274609691163 -0.0083209078602589, 0.0068355631775448 -0.0083371121598894, 0.0068544683305448 -0.0083344114432843, 0.0068544683305448 -0.0083074042772335)), ((0.0067950521354021 -0.0001539408464897, 0.0068166580245449 -0.0001620429963049, 0.0068274609691163 -0.0001431379800694, 0.0068085558161164 -0.0001323351136490, 0.0067950521354021 -0.0001539408464897)), ((0.0068247602329735 -0.0033353850072768, 0.0067869499269736 -0.0033488885903022, 0.0067923513992593 -0.0033785964729581, 0.0068247602329735 -0.0033353850072768)), ((0.0068031543438307 -0.0078563846041848, 0.0067599425655451 -0.0078698881872102, 0.0067464388848309 -0.0079049975030762, 0.0067950521354021 -0.0078860924868407, 0.0068031543438307 -0.0078563846041848)), ((0.0068004536076878 -0.0081210548314828, 0.0067653440378308 -0.0081237555480879, 0.0067491396209737 -0.0081561641473489, 0.0067896506631164 -0.0081534634307438, 0.0068004536076878 -0.0081210548314828)), ((0.0067977528715450 -0.0083047035606284, 0.0067248329956881 -0.0083020028440234, 0.0066978256342597 -0.0083425135930996, 0.0067626433016880 -0.0083317107266793, 0.0067545410932594 -0.0083695207591504, 0.0067761469824022 -0.0083749221923606, 0.0067977528715450 -0.0083587178927301, 0.0067977528715450 -0.0083047035606284)), ((0.0067545410932594 -0.0072325190684107, 0.0067491396209737 -0.0072622269510666, 0.0067734462462594 -0.0072676283842768, 0.0067788477185451 -0.0072973362669327, 0.0067950521354021 -0.0072730298174870, 0.0067545410932594 -0.0072325190684107)), ((0.0067923513992593 -0.0082398863621065, 0.0067680447739737 -0.0082344849288963, 0.0067599425655451 -0.0082560906617370, 0.0067923513992593 -0.0082398863621065)), ((0.0067653440378308 -0.0074215692307665, 0.0066951248981168 -0.0074377735303970, 0.0066438109114027 -0.0074890871458935, 0.0066438109114027 -0.0075133935953393, 0.0067275337318310 -0.0074944885791037, 0.0067653440378308 -0.0074539778300275, 0.0067653440378308 -0.0074215692307665)), ((0.0067221322595453 -0.0086747017355247, 0.0067032271065454 -0.0087206139178111, 0.0067383366764024 -0.0087314167842315, 0.0067491396209737 -0.0086909060351552, 0.0067221322595453 -0.0086747017355247)), ((0.0067356359402595 -0.0088151389989890, 0.0067167307872596 -0.0088475475982500, 0.0067383366764024 -0.0088421461650398, 0.0067356359402595 -0.0088151389989890)), ((0.0067086285788311 -0.0084370386742775, 0.0067032271065454 -0.0084532429739080, 0.0067275337318310 -0.0084559436905131, 0.0067086285788311 -0.0084370386742775)), ((0.0066870226896883 -0.0073513505990343, 0.0066870226896883 -0.0073675548986648, 0.0067140300511167 -0.0073648541820598, 0.0066870226896883 -0.0073513505990343)), ((0.0066411101752599 -0.0082776963945776, 0.0066681175366884 -0.0082993021274183, 0.0066681175366884 -0.0082776963945776, 0.0066411101752599 -0.0082776963945776)), ((0.0066222050222600 -0.0079779168514135, 0.0066357087029742 -0.0079914204344389, 0.0066492123836884 -0.0079725154182033, 0.0066222050222600 -0.0079779168514135)), ((0.0066465116475456 -0.0037756018139052, 0.0065870954524030 -0.0037918061135357, 0.0065519858825459 -0.0038539225954526, 0.0066465116475456 -0.0037756018139052)), ((0.0066357087029742 -0.0080319311835151, 0.0066032998692600 -0.0080508361997507, 0.0066303072306885 -0.0080508361997507, 0.0066357087029742 -0.0080319311835151)), ((0.0066141028138314 -0.0083263092934691, 0.0065843947162601 -0.0083398128764945, 0.0065897961885458 -0.0083911264919911, 0.0066357087029742 -0.0083722214757555, 0.0066141028138314 -0.0083263092934691)), ((0.0066141028138314 -0.0077213487739306, 0.0065762925078316 -0.0077321516403510, 0.0065816939801173 -0.0077672609562170, 0.0066195042861171 -0.0077564580897967, 0.0066141028138314 -0.0077213487739306)), ((0.0065708910355459 -0.0079698147015982, 0.0065654895632602 -0.0079860190012287, 0.0065978983969743 -0.0080022233008592, 0.0066060006054029 -0.0079806175680185, 0.0065708910355459 -0.0079698147015982)), ((0.0065654895632602 -0.0023118134139506, 0.0065708910355459 -0.0023361198633963, 0.0065951976608315 -0.0023253169969760, 0.0065654895632602 -0.0023118134139506)), ((0.0065465844102603 -0.0075782107938613, 0.0065600880909745 -0.0076052179599121, 0.0065924969246886 -0.0075944150934918, 0.0065465844102603 -0.0075782107938613)), ((0.0065816939801173 -0.0087179132012060, 0.0065573873548316 -0.0087125117679959, 0.0065465844102603 -0.0087341175008365, 0.0065816939801173 -0.0087179132012060)), ((0.0065654895632602 -0.0073513505990343, 0.0065060733681176 -0.0073675548986648, 0.0064736645344034 -0.0074269706639767, 0.0065573873548316 -0.0074053649311360, 0.0065654895632602 -0.0073513505990343)), ((0.0065330807295460 -0.0077456552233764, 0.0065249785211175 -0.0077780638226374, 0.0065438836741174 -0.0077699616728221, 0.0065330807295460 -0.0077456552233764)), ((0.0064925696874033 -0.0082236820624760, 0.0065114748404032 -0.0082641928115522, 0.0065411829379746 -0.0082533899451319, 0.0065141755765461 -0.0082209813458709, 0.0064925696874033 -0.0082236820624760)), ((0.0065276792572603 -0.0083587178927301, 0.0065033726319747 -0.0083695207591504, 0.0065006718958319 -0.0084181336580419, 0.0065330807295460 -0.0083938272085962, 0.0065276792572603 -0.0083587178927301)), ((0.0065222777849746 -0.0071785047363091, 0.0064979711596890 -0.0071677018698887, 0.0064655623259749 -0.0072028111857548, 0.0065222777849746 -0.0071785047363091)), ((0.0065087741042604 -0.0000432114656813, 0.0064925696874033 -0.0000837222147576, 0.0065168763126889 -0.0000756200649423, 0.0065087741042604 -0.0000432114656813)), ((0.0064844674789748 -0.0076241229761477, 0.0064655623259749 -0.0076349258425680, 0.0064520586452606 -0.0077051444743001, 0.0064844674789748 -0.0076241229761477)), ((0.0064844674789748 -0.0064412091031216, 0.0064628615898320 -0.0064709169857775, 0.0064844674789748 -0.0064682162691724, 0.0064844674789748 -0.0064412091031216)), ((0.0064817667428319 -0.0082776963945776, 0.0064628615898320 -0.0082749956779725, 0.0063818395055466 -0.0083560171761250, 0.0063737372971181 -0.0083965279252012, 0.0064844674789748 -0.0083128057104437, 0.0064817667428319 -0.0082776963945776)), ((0.0064142483392608 -0.0074188685141614, 0.0064061461308322 -0.0074755835628681, 0.0064709637982605 -0.0074917878624986, 0.0064142483392608 -0.0074188685141614)), ((0.0064412557006892 -0.0000162042996305, 0.0064061461308322 -0.0000216057328407, 0.0064061461308322 -0.0000513136154966, 0.0064412557006892 -0.0000162042996305)), ((0.0064061461308322 -0.0071677018698887, 0.0064142483392608 -0.0071947090359396, 0.0064412557006892 -0.0071893076027294, 0.0064412557006892 -0.0071677018698887, 0.0064061461308322 -0.0071677018698887)), ((0.0064115476031179 -0.0076511301421985, 0.0063899417139751 -0.0076457287089883, 0.0063818395055466 -0.0076700351584341, 0.0064115476031179 -0.0076511301421985)), ((0.0063548321441181 -0.0072298183518056, 0.0063170218381183 -0.0072568255178565, 0.0063197225742611 -0.0072838326839073, 0.0063656350886895 -0.0072514240846463, 0.0063548321441181 -0.0072298183518056)), ((0.0063278247826897 -0.0077375530735611, 0.0063305255188325 -0.0077645602396120, 0.0063494306718324 -0.0077645602396120, 0.0063521314079753 -0.0077456552233764, 0.0063278247826897 -0.0077375530735611)), ((0.0063494306718324 -0.0002943781099540, 0.0063359269911182 -0.0002808745269286, 0.0062873137405470 -0.0002997795431641, 0.0062657078514042 -0.0002835752435337, 0.0062765107959756 -0.0002484659276676, 0.0062387004899757 -0.0002511666442727, 0.0062170946008330 -0.0002889766767438, 0.0062414012261186 -0.0003294874258201, 0.0061873865032616 -0.0003429910088455, 0.0061765835586903 -0.0003186845593997, 0.0061333717804047 -0.0003105824095845, 0.0061252695719762 -0.0003645967416861, 0.0061603791418332 -0.0003672974582912, 0.0061765835586903 -0.0003970053409471, 0.0062224960731187 -0.0003970053409471, 0.0062576056429757 -0.0003672974582912, 0.0062765107959756 -0.0003078816929794, 0.0063278247826897 -0.0003240859926099, 0.0063494306718324 -0.0002943781099540)), ((0.0063224233104040 -0.0021173618183846, 0.0062873137405470 -0.0021281646848049, 0.0062873137405470 -0.0021470697010405, 0.0063224233104040 -0.0021173618183846)), ((0.0063143211019754 -0.0066626678647383, 0.0062846130044041 -0.0066977771806044, 0.0063197225742611 -0.0066842735975790, 0.0063143211019754 -0.0066626678647383)), ((0.0063170218381183 -0.0089474741126381, 0.0062657078514042 -0.0089609776956635, 0.0062657078514042 -0.0089852841451092, 0.0063116203658326 -0.0089771819952940, 0.0063170218381183 -0.0089474741126381)), ((0.0063143211019754 -0.0001944515955659, 0.0062873137405470 -0.0002025537453812, 0.0062819122682613 -0.0002295609114320, 0.0063116203658326 -0.0002187580450117, 0.0063143211019754 -0.0001944515955659)), ((0.0063089196296897 -0.0078806910536305, 0.0062684085875470 -0.0078752896204203, 0.0062603063791185 -0.0079130996528915, 0.0062954159489755 -0.0079103989362864, 0.0063089196296897 -0.0078806910536305)), ((0.0062468026984043 -0.0086855046019451, 0.0061981894478330 -0.0087179132012060, 0.0062170946008330 -0.0087341175008365, 0.0062008901839759 -0.0087665261000975, 0.0061657806141189 -0.0087584239502823, 0.0061468754611189 -0.0087881318329382, 0.0062251968092615 -0.0087908325495433, 0.0062278975454043 -0.0088475475982500, 0.0062630071152614 -0.0088772554809059, 0.0063008174212612 -0.0088070368491738, 0.0062549049068328 -0.0087719275333077, 0.0062468026984043 -0.0086855046019451)), ((0.0062846130044041 -0.0016123278132342, 0.0062035909201187 -0.0016204299630495, 0.0061765835586903 -0.0016420356958901, 0.0061630798779760 -0.0017014514612019, 0.0062008901839759 -0.0017257579106477, 0.0062630071152614 -0.0016987507445968, 0.0062981166851184 -0.0016582399955206, 0.0062846130044041 -0.0016123278132342)), ((0.0062900144766898 -0.0001566415630948, 0.0062387004899757 -0.0001566415630948, 0.0062197953369758 -0.0001917508789609, 0.0062630071152614 -0.0001917508789609, 0.0062900144766898 -0.0001566415630948)), ((0.0062387004899757 -0.0021848797335117, 0.0062792115321184 -0.0022037847497473, 0.0062711093236899 -0.0021821790169066, 0.0062387004899757 -0.0021848797335117)), ((0.0062684085875470 -0.0082533899451319, 0.0062197953369758 -0.0082533899451319, 0.0062062916562616 -0.0082939006942081, 0.0062522041706900 -0.0082830978277878, 0.0062684085875470 -0.0082533899451319)), ((0.0062603063791185 -0.0084883522897740, 0.0061927879755473 -0.0085369651886655, 0.0062089923924044 -0.0085423666218757, 0.0062603063791185 -0.0084883522897740)), ((0.0062495034345471 -0.0084262358078571, 0.0061981894478330 -0.0084235350912521, 0.0061954887116902 -0.0084586444071181, 0.0062414012261186 -0.0084586444071181, 0.0062495034345471 -0.0084262358078571)), ((0.0062143938646901 -0.0067247843466552, 0.0062251968092615 -0.0067571929459162, 0.0062359997538329 -0.0067463900794959, 0.0062143938646901 -0.0067247843466552)), ((0.0061927879755473 -0.0070974832381566, 0.0062008901839759 -0.0071541982868633, 0.0062305982815472 -0.0071514975702582, 0.0062224960731187 -0.0070893810883414, 0.0061927879755473 -0.0070974832381566)), ((0.0062062916562616 -0.0080076247340694, 0.0061819850309760 -0.0080076247340694, 0.0061711820864046 -0.0080454347665405, 0.0062062916562616 -0.0080346319001202, 0.0062062916562616 -0.0080076247340694)), ((0.0061684813502617 -0.0078860924868407, 0.0061603791418332 -0.0079049975030762, 0.0061765835586903 -0.0079076982196813, 0.0061684813502617 -0.0078860924868407)), ((0.0061684813502617 -0.0002376630612473, 0.0061522769334046 -0.0002673709439032, 0.0061765835586903 -0.0002646702272981, 0.0061684813502617 -0.0002376630612473)), ((0.0061333717804047 -0.0067409886462857, 0.0061387732526904 -0.0067706965289416, 0.0061711820864046 -0.0067652950957315, 0.0061711820864046 -0.0067436893628908, 0.0061333717804047 -0.0067409886462857)), ((0.0061657806141189 -0.0006184641025639, 0.0060280430708337 -0.0006211648191690, 0.0060118386539766 -0.0006346684021944, 0.0060226415985480 -0.0006778798678757, 0.0060550504322622 -0.0006805805844808, 0.0060550504322622 -0.0006562741350350, 0.0061009629466906 -0.0006319676855893, 0.0061117658912619 -0.0006643762848503, 0.0061468754611189 -0.0006670770014554, 0.0061657806141189 -0.0006184641025639)), ((0.0061657806141189 -0.0021335661180151, 0.0061333717804047 -0.0021065589519643, 0.0061009629466906 -0.0021092596685694, 0.0060577511684050 -0.0021551718508558, 0.0060334445431194 -0.0022361933490082, 0.0061468754611189 -0.0021983833165371, 0.0061657806141189 -0.0021335661180151)), ((0.0061630798779760 -0.0085018558727995, 0.0061036636828334 -0.0085180601724299, 0.0060766563214049 -0.0085639723547164, 0.0061198680996905 -0.0085612716381113, 0.0061630798779760 -0.0085018558727995)), ((0.0060334445431194 -0.0000702186317321, 0.0061090651551191 -0.0000999265143880, 0.0061495761972618 -0.0000486128988915, 0.0060793570575478 -0.0000378100324712, 0.0060334445431194 -0.0000702186317321)), ((0.0061333717804047 -0.0078806910536305, 0.0061117658912619 -0.0078725889038153, 0.0060928607382620 -0.0078941946366559, 0.0061333717804047 -0.0078806910536305)), ((0.0061252695719762 -0.0001755465793304, 0.0060388460154051 -0.0001998530287761, 0.0060388460154051 -0.0002295609114320, 0.0059794298202625 -0.0002781738103235, 0.0059713276118339 -0.0003402902922404, 0.0060496489599765 -0.0003618960250810, 0.0060037364455481 -0.0003132831261896, 0.0060280430708337 -0.0002835752435337, 0.0060550504322622 -0.0002862759601387, 0.0060442474876908 -0.0002592687940879, 0.0061252695719762 -0.0001755465793304)), ((0.0060334445431194 -0.0004375160900234, 0.0060550504322622 -0.0004834282723098, 0.0061252695719762 -0.0004861289889148, 0.0061117658912619 -0.0004267132236030, 0.0060334445431194 -0.0004375160900234)), ((0.0061225688358333 -0.0001323351136490, 0.0060604519045479 -0.0001269336804389, 0.0059821305564053 -0.0001809480125405, 0.0060307438069765 -0.0001890501623558, 0.0061225688358333 -0.0001323351136490)), ((0.0061117658912619 -0.0002781738103235, 0.0060766563214049 -0.0002781738103235, 0.0060658533768335 -0.0003186845593997, 0.0061198680996905 -0.0002997795431641, 0.0061117658912619 -0.0002781738103235)), ((0.0061171673635476 -0.0023820320456827, 0.0060847585298335 -0.0023847327622878, 0.0060874592659763 -0.0024009370619183, 0.0061171673635476 -0.0023820320456827)), ((0.0060820577936906 -0.0081588648639540, 0.0060712548491192 -0.0081885727466099, 0.0060955614744049 -0.0082047770462404, 0.0061144666274048 -0.0081777698801896, 0.0060820577936906 -0.0081588648639540)), ((0.0060442474876908 -0.0079779168514135, 0.0060874592659763 -0.0079779168514135, 0.0060982622105477 -0.0079536104019677, 0.0060577511684050 -0.0079455082521525, 0.0060442474876908 -0.0079779168514135)), ((0.0060874592659763 -0.0005050340051504, 0.0060415467515479 -0.0005752526368826, 0.0060928607382620 -0.0005536469040419, 0.0060874592659763 -0.0005050340051504)), ((0.0060550504322622 -0.0067247843466552, 0.0059902327648338 -0.0067382879296806, 0.0059794298202625 -0.0067950029783874, 0.0060226415985480 -0.0068220101444382, 0.0060172401262623 -0.0068571194603043, 0.0060361452792622 -0.0068571194603043, 0.0060901600021192 -0.0067625943791264, 0.0060550504322622 -0.0067247843466552)), ((0.0060361452792622 -0.0063817933378098, 0.0060226415985480 -0.0064033990706504, 0.0060442474876908 -0.0064358076699114, 0.0060361452792622 -0.0063817933378098)), ((0.0060172401262623 -0.0068868273429602, 0.0060091379178338 -0.0069138345090110, 0.0060307438069765 -0.0069219366588262, 0.0060415467515479 -0.0068949294927754, 0.0060172401262623 -0.0068868273429602)), ((0.0060064371816909 -0.0061468309931676, 0.0060037364455481 -0.0061684367260082, 0.0060280430708337 -0.0061765388758235, 0.0060361452792622 -0.0061522324263778, 0.0060064371816909 -0.0061468309931676)), ((0.0060226415985480 -0.0005185375881758, 0.0059524224588340 -0.0005293404545962, 0.0059443202504055 -0.0005617490538571, 0.0059875320286910 -0.0005698512036724, 0.0060226415985480 -0.0005185375881758)), ((0.0059902327648338 -0.0079941211510440, 0.0060064371816909 -0.0080049240174643, 0.0060226415985480 -0.0079860190012287, 0.0059902327648338 -0.0079941211510440)), ((0.0059902327648338 -0.0000054014332102, 0.0059632254034054 -0.0000108028664203, 0.0059470209865483 -0.0000783207815474, 0.0059011084721199 -0.0001269336804389, 0.0060118386539766 -0.0000729193483372, 0.0059902327648338 -0.0000054014332102)), ((0.0060064371816909 -0.0044129709327047, 0.0059524224588340 -0.0044345766655453, 0.0059443202504055 -0.0044912917142521, 0.0059929335009767 -0.0044696859814114, 0.0060064371816909 -0.0044129709327047)), ((0.0059794298202625 -0.0003753996081065, 0.0059875320286910 -0.0004024067741573, 0.0059983349732624 -0.0003835017579217, 0.0059794298202625 -0.0003753996081065)), ((0.0059848312925482 -0.0070191624566092, 0.0059443202504055 -0.0070542717724753, 0.0059794298202625 -0.0070488703392651, 0.0059848312925482 -0.0070191624566092)), ((0.0059740283479768 -0.0064358076699114, 0.0059632254034054 -0.0064736177023826, 0.0059848312925482 -0.0064628148359622, 0.0059740283479768 -0.0064358076699114)), ((0.0059281158335484 -0.0065654420669554, 0.0059146121528341 -0.0065951499496113, 0.0059335173058341 -0.0066005513828214, 0.0059443202504055 -0.0065816463665859, 0.0059281158335484 -0.0065654420669554)), ((0.0059281158335484 -0.0000459121822864, 0.0058227871239774 -0.0000567150487067, 0.0058389915408344 -0.0000972257977830, 0.0058119841794060 -0.0001458386966745, 0.0058632981661201 -0.0001620429963049, 0.0058741011106914 -0.0002025537453812, 0.0059443202504055 -0.0001728458627253, 0.0059416195142626 -0.0001512401298846, 0.0058551959576915 -0.0001404372634643, 0.0058768018468343 -0.0000999265143880, 0.0059335173058341 -0.0000648171985220, 0.0059281158335484 -0.0000459121822864)), ((0.0058957069998342 -0.0044264745157301, 0.0059119114166913 -0.0044561823983860, 0.0059146121528341 -0.0044318759489402, 0.0058957069998342 -0.0044264745157301)), ((0.0058335900685487 -0.0061387288433523, 0.0058389915408344 -0.0061684367260082, 0.0058984077359771 -0.0061657360094032, 0.0058659989022629 -0.0061576338595879, 0.0058524952215487 -0.0061252252603269, 0.0058335900685487 -0.0061387288433523)), ((0.0058849040552628 -0.0032030498936277, 0.0058443930131201 -0.0032192541932582, 0.0058416922769773 -0.0032570642257294, 0.0058930062636914 -0.0032381592094938, 0.0058849040552628 -0.0032030498936277)), ((0.0058903055275485 -0.0007264927667672, 0.0058659989022629 -0.0007345949165824, 0.0058741011106914 -0.0007724049490536, 0.0058903055275485 -0.0007264927667672)), ((0.0058741011106914 -0.0035325373194478, 0.0057903782902632 -0.0035298366028427, 0.0057687724011204 -0.0035838509349444, 0.0058200863878345 -0.0035892523681545, 0.0058768018468343 -0.0035595444854986, 0.0058741011106914 -0.0035325373194478)), ((0.0058578966938344 -0.0063169761392878, 0.0058443930131201 -0.0063682897547843, 0.0058632981661201 -0.0063574868883640, 0.0058578966938344 -0.0063169761392878)), ((0.0057795753456918 -0.0065330334676944, 0.0057930790264060 -0.0065519384839299, 0.0058119841794060 -0.0065384349009045, 0.0057795753456918 -0.0065330334676944)), ((0.0057768746095490 -0.0001134300974135, 0.0057417650396920 -0.0001431379800694, 0.0057768746095490 -0.0001377365468592, 0.0057768746095490 -0.0001134300974135)), ((0.0057444657758348 -0.0068436158772789, 0.0057336628312634 -0.0068274115776484, 0.0057147576782635 -0.0068517180270941, 0.0057444657758348 -0.0068436158772789)), ((0.0057228598866920 -0.0069651481245076, 0.0057120569421206 -0.0069813524241381, 0.0057363635674063 -0.0070029581569787, 0.0057444657758348 -0.0069786517075330, 0.0057228598866920 -0.0069651481245076)), ((0.0057174584144063 -0.0004348153734183, 0.0056823488445493 -0.0004510196730488, 0.0056769473722636 -0.0004969318553352, 0.0057174584144063 -0.0004888297055199, 0.0057174584144063 -0.0004348153734183)), ((0.0056256333855495 -0.0067463900794959, 0.0056337355939781 -0.0067814993953620, 0.0056877503168350 -0.0067923022617823, 0.0056256333855495 -0.0067463900794959)), ((0.0056661444276922 -0.0069624474079025, 0.0055932245518354 -0.0069651481245076, 0.0055932245518354 -0.0069840531407431, 0.0056148304409782 -0.0069921552905584, 0.0056661444276922 -0.0069624474079025)), ((0.0056283341216924 -0.0001647437129100, 0.0055824216072640 -0.0001755465793304, 0.0055689179265498 -0.0002079551785913, 0.0056229326494067 -0.0001917508789609, 0.0056283341216924 -0.0001647437129100)), ((0.0056148304409782 -0.0003753996081065, 0.0055851223434068 -0.0003808010413166, 0.0055851223434068 -0.0004132096405776, 0.0056229326494067 -0.0003997060575522, 0.0056148304409782 -0.0003753996081065)), ((0.0055851223434068 -0.0032516627925192, 0.0055527135096927 -0.0032489620759142, 0.0055068009952643 -0.0032840713917802, 0.0055176039398357 -0.0033137792744361, 0.0055554142458355 -0.0033056771246209, 0.0055851223434068 -0.0032516627925192)), ((0.0055392098289785 -0.0049423113873008, 0.0054959980506929 -0.0049855228529821, 0.0054959980506929 -0.0050152307356381, 0.0055554142458355 -0.0049801214197720, 0.0055392098289785 -0.0049423113873008)), ((0.0055203046759785 -0.0000270071660508, 0.0054905965784072 -0.0000324085992610, 0.0054770928976930 -0.0000675179151271, 0.0055284068844071 -0.0000594157653118, 0.0055203046759785 -0.0000270071660508)), ((0.0054959980506929 -0.0045128974470927, 0.0054824943699787 -0.0045264010301181, 0.0055013995229786 -0.0045453060463537, 0.0055122024675500 -0.0045237003135131, 0.0054959980506929 -0.0045128974470927)), ((0.0053933700772647 -0.0046128239614808, 0.0053798663965505 -0.0046614368603723, 0.0054014722856933 -0.0046884440264231, 0.0054959980506929 -0.0046965461762383, 0.0053933700772647 -0.0046128239614808)), ((0.0054635892169787 -0.0054338418094258, 0.0054581877446931 -0.0054527468256614, 0.0054824943699787 -0.0054392432426360, 0.0054635892169787 -0.0054338418094258)), ((0.0054284796471217 -0.0001080286642033, 0.0054473848001217 -0.0001296343970440, 0.0054689906892644 -0.0001215322472287, 0.0054284796471217 -0.0001080286642033)), ((0.0054230781748360 -0.0001944515955659, 0.0053960708134076 -0.0001998530287761, 0.0053987715495504 -0.0002403637778523, 0.0054338811194074 -0.0002241594782218, 0.0054230781748360 -0.0001944515955659)), ((0.0053906693411219 -0.0085882788041621, 0.0054149759664075 -0.0086341909864485, 0.0054230781748360 -0.0086044831037926, 0.0053906693411219 -0.0085882788041621)), ((0.0054095744941218 -0.0001458386966745, 0.0053744649242648 -0.0001458386966745, 0.0053636619796934 -0.0001809480125405, 0.0053933700772647 -0.0001782472959354, 0.0054095744941218 -0.0001458386966745)), ((0.0053663627158363 -0.0047235533422892, 0.0053231509375507 -0.0047505605083400, 0.0053258516736936 -0.0047775676743908, 0.0053717641881220 -0.0047586626581552, 0.0053663627158363 -0.0047235533422892)), ((0.0053717641881220 -0.0053663238942988, 0.0053447568266935 -0.0054149367931902, 0.0053690634519791 -0.0054095353599801, 0.0053717641881220 -0.0053663238942988)), ((0.0053447568266935 -0.0044858902810419, 0.0053393553544078 -0.0045291017467232, 0.0053555597712649 -0.0045318024633283, 0.0053609612435506 -0.0044939924308571, 0.0053447568266935 -0.0044858902810419)), ((0.0052745376869795 -0.0031301305452905, 0.0052853406315509 -0.0031571377113413, 0.0053258516736936 -0.0031382326951058, 0.0053177494652650 -0.0031220283954753, 0.0052745376869795 -0.0031301305452905)), ((0.0052826398954080 -0.0051016536670007, 0.0052286251725511 -0.0051070551002109, 0.0052286251725511 -0.0051394636994718, 0.0052826398954080 -0.0051016536670007)), ((0.0052070192834083 -0.0046587361437672, 0.0051773111858370 -0.0046560354271621, 0.0051151942545515 -0.0046992468928434, 0.0051070920461230 -0.0047424583585247, 0.0051881141304084 -0.0047073490426587, 0.0052070192834083 -0.0046587361437672)), ((0.0051503038244085 -0.0053744260441140, 0.0051422016159800 -0.0053960317769547, 0.0051638075051228 -0.0054041339267699, 0.0051503038244085 -0.0053744260441140)), ((0.0051259971991229 -0.0058821607658695, 0.0051043913099801 -0.0059037664987102, 0.0051286979352658 -0.0059010657821051, 0.0051259971991229 -0.0058821607658695)), ((0.0051043913099801 -0.0047802683909959, 0.0050719824762660 -0.0048045748404416, 0.0051070920461230 -0.0047964726906264, 0.0051043913099801 -0.0047802683909959)), ((0.0050746832124088 -0.0052366894972548, 0.0050395736425518 -0.0052447916470701, 0.0050125662811234 -0.0053015066957768, 0.0050908876292659 -0.0052799009629361, 0.0050746832124088 -0.0052366894972548)), ((0.0050179677534091 -0.0068544187436992, 0.0050314714341233 -0.0068733237599348, 0.0050395736425518 -0.0068544187436992, 0.0050179677534091 -0.0068544187436992)), ((0.0049666537666950 -0.0071785047363091, 0.0049855589196949 -0.0072514240846463, 0.0050152670172662 -0.0072487233680412, 0.0050017633365520 -0.0071893076027294, 0.0049666537666950 -0.0071785047363091)), ((0.0049477486136950 -0.0035190337364224, 0.0049045368354095 -0.0035406394692631, 0.0049072375715523 -0.0036594709998867, 0.0049855589196949 -0.0036054566677850, 0.0049666537666950 -0.0035784495017342, 0.0049126390438380 -0.0035838509349444, 0.0049477486136950 -0.0035190337364224)), ((0.0048316169595526 -0.0037242881984087, 0.0048721280016953 -0.0037134853319883, 0.0048478213764097 -0.0036945803157527, 0.0048316169595526 -0.0037242881984087)), ((0.0046263610126963 -0.0034245086552445, 0.0045642440814108 -0.0034758222707411, 0.0045885507066964 -0.0034974280035817, 0.0045696455536965 -0.0035190337364224, 0.0045777477621251 -0.0035595444854986, 0.0045453389284109 -0.0035568437688935, 0.0045426381922680 -0.0035109315866071, 0.0045102293585539 -0.0035217344530275, 0.0044940249416968 -0.0035811502183393, 0.0045210323031253 -0.0036297631172308, 0.0044805212609826 -0.0036729745829121, 0.0045264337754110 -0.0036594709998867, 0.0045507404006966 -0.0036189602508104, 0.0046047551235535 -0.0036162595342054, 0.0046452661656962 -0.0037242881984087, 0.0045750470259822 -0.0037296896316188, 0.0045723462898394 -0.0037674996640900, 0.0045939521789821 -0.0037864046803256, 0.0046533683741248 -0.0037485946478544, 0.0046911786801246 -0.0037647989474849, 0.0047397919306959 -0.0037539960810646, 0.0047451934029815 -0.0037107846153832, 0.0046857772078389 -0.0036810767327273, 0.0046749742632675 -0.0036351645504409, 0.0047640985559815 -0.0035865516515495, 0.0048370184318383 -0.0035973545179698, 0.0048505221125526 -0.0035730480685240, 0.0048424199041240 -0.0035460409024732, 0.0047938066535528 -0.0035487416190783, 0.0046965801524103 -0.0036189602508104, 0.0046371639572677 -0.0035973545179698, 0.0046263610126963 -0.0035514423356834, 0.0046695727909818 -0.0035082308700021, 0.0046047551235535 -0.0034785229873462, 0.0046263610126963 -0.0034245086552445)), ((0.0048316169595526 -0.0054473453924512, 0.0048046095981242 -0.0054608489754767, 0.0048289162234098 -0.0054689511252919, 0.0048316169595526 -0.0054473453924512)), ((0.0047695000282672 -0.0034164065054293, 0.0046992808885532 -0.0034380122382699, 0.0046992808885532 -0.0034785229873462, 0.0047613978198386 -0.0034704208375309, 0.0047695000282672 -0.0034164065054293)), ((0.0046857772078389 -0.0031922470272074, 0.0046344632211248 -0.0032273563430735, 0.0046749742632675 -0.0032192541932582, 0.0046857772078389 -0.0031922470272074)), ((0.0046641713186961 -0.0075863129436765, 0.0046128573319821 -0.0076052179599121, 0.0046641713186961 -0.0076079186765172, 0.0046641713186961 -0.0075863129436765)), ((0.0046641713186961 -0.0030572111969533, 0.0045831492344107 -0.0030707147799787, 0.0045804484982679 -0.0031193276788702, 0.0046587698464105 -0.0030896197962143, 0.0046641713186961 -0.0030572111969533)), ((0.0046074558596964 -0.0058416500167933, 0.0046344632211248 -0.0058524528832136, 0.0046560691102676 -0.0058335478669780, 0.0046128573319821 -0.0058254457171628, 0.0046074558596964 -0.0058416500167933)), ((0.0046425654295534 -0.0057390227858001, 0.0045939521789821 -0.0058038399843221, 0.0046263610126963 -0.0057849349680865, 0.0046425654295534 -0.0057390227858001)), ((0.0046290617488391 -0.0059010657821051, 0.0046074558596964 -0.0059064672153153, 0.0046101565958392 -0.0059280729481559, 0.0046290617488391 -0.0059010657821051)), ((0.0045885507066964 -0.0029275767999093, 0.0045480396645537 -0.0029410803829348, 0.0045669448175537 -0.0029626861157754, 0.0045912514428393 -0.0029545839659602, 0.0045885507066964 -0.0029275767999093)), ((0.0045804484982679 -0.0032408599260989, 0.0045615433452680 -0.0032597649423345, 0.0045723462898394 -0.0032732685253599, 0.0045318352476967 -0.0032921735415955, 0.0045210323031253 -0.0033353850072768, 0.0045831492344107 -0.0032921735415955, 0.0045804484982679 -0.0032408599260989)), ((0.0045696455536965 -0.0051907773149684, 0.0045426381922680 -0.0052042808979938, 0.0045561418729823 -0.0052366894972548, 0.0045777477621251 -0.0052285873474396, 0.0045696455536965 -0.0051907773149684)), ((0.0044967256778397 -0.0032246556264684, 0.0044130028574114 -0.0032192541932582, 0.0043886962321258 -0.0032813706751751, 0.0043643896068402 -0.0032921735415955, 0.0043562873984116 -0.0033515893069073, 0.0043967984405543 -0.0033083778412260, 0.0044481124272684 -0.0033245821408565, 0.0044967256778397 -0.0032246556264684)), ((0.0044886234694111 -0.0031031233792397, 0.0044427109549827 -0.0031031233792397, 0.0044427109549827 -0.0031328312618956, 0.0044859227332683 -0.0031328312618956, 0.0044886234694111 -0.0031031233792397)), ((0.0044184043296971 -0.0034056036390089, 0.0044886234694111 -0.0033866986227734, 0.0044292072742685 -0.0033839979061683, 0.0044184043296971 -0.0034056036390089)), ((0.0044427109549827 -0.0021740768670913, 0.0044238058019828 -0.0022172883327727, 0.0044805212609826 -0.0022280911991930, 0.0044859227332683 -0.0021929818833269, 0.0044427109549827 -0.0021740768670913)), ((0.0044535138995541 -0.0000027007166051, 0.0044670175802683 -0.0000378100324712, 0.0044778205248397 -0.0000081021498152, 0.0044535138995541 -0.0000027007166051)), ((0.0042968712032690 -0.0030221018810872, 0.0042995719394118 -0.0030761162131889, 0.0043508859261259 -0.0031328312618956, 0.0044103021212686 -0.0030950212294245, 0.0043913969682686 -0.0030275033142974, 0.0043562873984116 -0.0030599119135584, 0.0042968712032690 -0.0030221018810872)), ((0.0043886962321258 -0.0053879296271394, 0.0043805940236972 -0.0054500461090563, 0.0044021999128400 -0.0054338418094258, 0.0043886962321258 -0.0053879296271394)), ((0.0043157763562689 -0.0034623186877157, 0.0042995719394118 -0.0034893258537665, 0.0043292800369832 -0.0035001287201868, 0.0043427837176974 -0.0034704208375309, 0.0043157763562689 -0.0034623186877157)), ((0.0043265793008403 -0.0068112072780179, 0.0042725645779834 -0.0068166087112280, 0.0042509586888406 -0.0068571194603043, 0.0043265793008403 -0.0068112072780179)), ((0.0043184770924118 -0.0079995225842541, 0.0042995719394118 -0.0079806175680185, 0.0042563601611263 -0.0080049240174643, 0.0042995719394118 -0.0080184276004897, 0.0043184770924118 -0.0079995225842541)), ((0.0042590608972691 -0.0032057506102328, 0.0042806667864119 -0.0032300570596786, 0.0043157763562689 -0.0032219549098633, 0.0043022726755547 -0.0032030498936277, 0.0042590608972691 -0.0032057506102328)), ((0.0042914697309833 -0.0001026272309931, 0.0042293527996978 -0.0001323351136490, 0.0042158491189836 -0.0001782472959354, 0.0041672358684124 -0.0001728458627253, 0.0041861410214123 -0.0002268601948269, 0.0042752653141262 -0.0002349623446422, 0.0042779660502691 -0.0001728458627253, 0.0043076741478404 -0.0001296343970440, 0.0042914697309833 -0.0001026272309931)), ((0.0042077469105551 -0.0053771267607191, 0.0042131483828408 -0.0054014332101648, 0.0042563601611263 -0.0054122360765852, 0.0042617616334120 -0.0053852289105343, 0.0042347542719835 -0.0053960317769547, 0.0042077469105551 -0.0053771267607191)), ((0.0042509586888406 -0.0027709352368146, 0.0041969439659837 -0.0027844388198400, 0.0041969439659837 -0.0028114459858908, 0.0041645351322695 -0.0028276502855213, 0.0041780388129838 -0.0028681610345975, 0.0041051189371269 -0.0029005696338585, 0.0041078196732697 -0.0029248760833043, 0.0041402285069839 -0.0029032703504636, 0.0042104476466979 -0.0029032703504636, 0.0042374550081264 -0.0028600588847823, 0.0042158491189836 -0.0028168474191010, 0.0042509586888406 -0.0027709352368146)), ((0.0042455572165549 -0.0056498991378324, 0.0042212505912693 -0.0056498991378324, 0.0042158491189836 -0.0056742055872782, 0.0042455572165549 -0.0056498991378324)), ((0.0041861410214123 -0.0029653868323805, 0.0040997174648412 -0.0030004961482466, 0.0041348270346982 -0.0029896932818262, 0.0041753380768409 -0.0030356054641126, 0.0040970167286984 -0.0031112255290549, 0.0040646078949842 -0.0031139262456600, 0.0041024182009840 -0.0031652398611566, 0.0041726373406981 -0.0031436341283159, 0.0042320535358407 -0.0030356054641126, 0.0042158491189836 -0.0030167004478771, 0.0042266520635550 -0.0029923939984313, 0.0041861410214123 -0.0029653868323805)), ((0.0041915424936980 -0.0000594157653118, 0.0041510314515553 -0.0000594157653118, 0.0041159218816983 -0.0001080286642033, 0.0041888417575551 -0.0000864229313626, 0.0041915424936980 -0.0000594157653118)), ((0.0041132211455554 -0.0078725889038153, 0.0041429292431267 -0.0079185010861017, 0.0041861410214123 -0.0079103989362864, 0.0041132211455554 -0.0078725889038153)), ((0.0040970167286984 -0.0030437076139279, 0.0040970167286984 -0.0030599119135584, 0.0041267248262697 -0.0030545104803482, 0.0041240240901268 -0.0030383061807177, 0.0040970167286984 -0.0030437076139279)), ((0.0040862137841270 -0.0053852289105343, 0.0041186226178411 -0.0053825281939293, 0.0040970167286984 -0.0053690246109038, 0.0040862137841270 -0.0053852289105343)), ((0.0040970167286984 -0.0001863494457507, 0.0040592064226985 -0.0002160573284066, 0.0040970167286984 -0.0002133566118015, 0.0040970167286984 -0.0001863494457507)), ((0.0040673086311270 -0.0062710639570014, 0.0040213961166986 -0.0062656625237912, 0.0039943887552702 -0.0063061732728674, 0.0040673086311270 -0.0062710639570014)), ((0.0040430020058414 -0.0050989529503956, 0.0040294983251272 -0.0051151572500261, 0.0040538049504128 -0.0051340622662617, 0.0040673086311270 -0.0051124565334210, 0.0040430020058414 -0.0050989529503956)), ((0.0039484762408418 -0.0029545839659602, 0.0039322718239847 -0.0030031968648517, 0.0039511769769846 -0.0030248025976923, 0.0039862865468416 -0.0030221018810872, 0.0039808850745559 -0.0030815176463990, 0.0040024909636987 -0.0030842183630041, 0.0040673086311270 -0.0029761896988008, 0.0039889872829845 -0.0030085982980618, 0.0039484762408418 -0.0029545839659602)), ((0.0036648989458429 -0.0027466287873688, 0.0036757018904143 -0.0027655338036044, 0.0037162129325570 -0.0027412273541587, 0.0037135121964141 -0.0027871395364451, 0.0037999357529852 -0.0027763366700247, 0.0038458482674136 -0.0028033438360756, 0.0039322718239847 -0.0027844388198400, 0.0039430747685561 -0.0028114459858908, 0.0038890600456992 -0.0028654603179924, 0.0039511769769846 -0.0028465553017569, 0.0039646806576989 -0.0029005696338585, 0.0039997902275559 -0.0029005696338585, 0.0040592064226985 -0.0028519567349670, 0.0040592064226985 -0.0028222488523111, 0.0040348997974129 -0.0028249495689162, 0.0040240968528415 -0.0027925409696552, 0.0039700821299846 -0.0027709352368146, 0.0039808850745559 -0.0027223223379231, 0.0038107386975566 -0.0027331252043434, 0.0038269431144137 -0.0026737094390316, 0.0039079651986991 -0.0026575051394011, 0.0039106659348419 -0.0026250965401401, 0.0038836585734135 -0.0026061915239045, 0.0038134394336995 -0.0026683080058214, 0.0037918335445567 -0.0026656072892163, 0.0037702276554139 -0.0027088187548977, 0.0037270158771284 -0.0027250230545282, 0.0037216144048427 -0.0026845123054519, 0.0036648989458429 -0.0027466287873688), (0.0037999357529852 -0.0027412273541587, 0.0037837313361281 -0.0027601323703942, 0.0037513225024140 -0.0027547309371841, 0.0037999357529852 -0.0027412273541587)), ((0.0040240968528415 -0.0077510566565865, 0.0039862865468416 -0.0077618595230069, 0.0039889872829845 -0.0077780638226374, 0.0040240968528415 -0.0077510566565865)), ((0.0040051916998416 -0.0030977219460295, 0.0039970894914130 -0.0031193276788702, 0.0040132939082701 -0.0031247291120804, 0.0040213961166986 -0.0031031233792397, 0.0040051916998416 -0.0030977219460295)), ((0.0039916880191273 -0.0059199707983407, 0.0039619799215560 -0.0059145693651305, 0.0039430747685561 -0.0059496786809966, 0.0039916880191273 -0.0059523793976017, 0.0039916880191273 -0.0059199707983407)), ((0.0039214688794133 -0.0001944515955659, 0.0039214688794133 -0.0002295609114320, 0.0039511769769846 -0.0002457652110625, 0.0039835858106988 -0.0002322616280371, 0.0039781843384131 -0.0002160573284066, 0.0039457755046989 -0.0002295609114320, 0.0039511769769846 -0.0001917508789609, 0.0039214688794133 -0.0001944515955659)), ((0.0037756291276996 -0.0023577255962370, 0.0037864320722710 -0.0024306449445742, 0.0037540232385568 -0.0024495499608098, 0.0038242423782708 -0.0024738564102555, 0.0037999357529852 -0.0025197685925419, 0.0039565784492703 -0.0024981628597012, 0.0039565784492703 -0.0024684549770453, 0.0038593519481278 -0.0024684549770453, 0.0038053372252709 -0.0023496234464217, 0.0037756291276996 -0.0023577255962370)), ((0.0039376732962704 -0.0077105459075103, 0.0039106659348419 -0.0076997430410900, 0.0038917607818420 -0.0077483559399815, 0.0039376732962704 -0.0077105459075103)), ((0.0039025637264134 -0.0025251700257521, 0.0038539504758422 -0.0025440750419876, 0.0038566512119850 -0.0025791843578537, 0.0039052644625562 -0.0025521771918029, 0.0039025637264134 -0.0025251700257521)), ((0.0038863593095563 -0.0029437810995398, 0.0038404467951279 -0.0029815911320110, 0.0038674541565564 -0.0030680140633736, 0.0038998629902705 -0.0030139997312720, 0.0038863593095563 -0.0029437810995398)), ((0.0038323445866994 -0.0022713026648743, 0.0038134394336995 -0.0023145141305556, 0.0038404467951279 -0.0023253169969760, 0.0038620526842707 -0.0023010105475302, 0.0038323445866994 -0.0022713026648743)), ((0.0037756291276996 -0.0028249495689162, 0.0037486217662711 -0.0028492560183620, 0.0037729283915568 -0.0028789639010179, 0.0037621254469854 -0.0029059710670687, 0.0038107386975566 -0.0029518832493551, 0.0038296438505565 -0.0028600588847823, 0.0037756291276996 -0.0028249495689162)), ((0.0038269431144137 -0.0025467757585927, 0.0037756291276996 -0.0025845857910639, 0.0037702276554139 -0.0026277972567452, 0.0038161401698423 -0.0026034908072995, 0.0038269431144137 -0.0025467757585927)), ((0.0038080379614138 -0.0030599119135584, 0.0037999357529852 -0.0030869190796092, 0.0038161401698423 -0.0030815176463990, 0.0038080379614138 -0.0030599119135584)), ((0.0037729283915568 -0.0029815911320110, 0.0037648261831282 -0.0030139997312720, 0.0037999357529852 -0.0030167004478771, 0.0038026364891281 -0.0029869925652212, 0.0037729283915568 -0.0029815911320110)), ((0.0037837313361281 -0.0022415947822184, 0.0037351180855569 -0.0022875069645048, 0.0037486217662711 -0.0023253169969760, 0.0037918335445567 -0.0023037112641353, 0.0037837313361281 -0.0022415947822184)), ((0.0037270158771284 -0.0005509461874368, 0.0036757018904143 -0.0005941576531181, 0.0036757018904143 -0.0006427705520096, 0.0036459937928430 -0.0006643762848503, 0.0036432930567001 -0.0007021863173214, 0.0036784026265571 -0.0006967848841113, 0.0037540232385568 -0.0005779533534876, 0.0037270158771284 -0.0005509461874368)), ((0.0037351180855569 -0.0023550248796319, 0.0037054099879856 -0.0023442220132115, 0.0036486945289858 -0.0023874334788929, 0.0036513952651287 -0.0024090392117335, 0.0036919063072714 -0.0024117399283386, 0.0037351180855569 -0.0023550248796319)), ((0.0036865048349857 -0.0024576521106250, 0.0036594974735572 -0.0024954621430962, 0.0036973077795570 -0.0025413743253826, 0.0037216144048427 -0.0025224693091470, 0.0037243151409855 -0.0024711556936504, 0.0036865048349857 -0.0024576521106250)), ((0.0036811033627000 -0.0025548779084080, 0.0036675996819857 -0.0026169943903249, 0.0037189136686998 -0.0026386001231655, 0.0037243151409855 -0.0026142936737198, 0.0036865048349857 -0.0026223958235350, 0.0037054099879856 -0.0025710822080385, 0.0036811033627000 -0.0025548779084080)), ((0.0037081107241284 -0.0022496969320337, 0.0036919063072714 -0.0022956091143201, 0.0037189136686998 -0.0022794048146896, 0.0037081107241284 -0.0022496969320337)), ((0.0036757018904143 -0.0058659564662390, 0.0036459937928430 -0.0058740586160543, 0.0036432930567001 -0.0059334743813661, 0.0036784026265571 -0.0059226715149457, 0.0036757018904143 -0.0058659564662390)), ((0.0036540960012715 -0.0021146611017795, 0.0036081834868431 -0.0021281646848049, 0.0035676724447004 -0.0021902811667218, 0.0036162856952716 -0.0021929818833269, 0.0036621982097000 -0.0021416682678304, 0.0036540960012715 -0.0021146611017795)), ((0.0036567967374143 -0.0023118134139506, 0.0036351908482716 -0.0023226162803709, 0.0036513952651287 -0.0023469227298166, 0.0036567967374143 -0.0023118134139506)), ((0.0035082562495578 -0.0026358994065604, 0.0035082562495578 -0.0026548044227960, 0.0035487672917005 -0.0026629065726113, 0.0035487672917005 -0.0026953151718723, 0.0035811761254146 -0.0027034173216875, 0.0035244606664149 -0.0027304244877383, 0.0035460665555576 -0.0027628330869993, 0.0036324901121287 -0.0027736359534196, 0.0036432930567001 -0.0026899137386621, 0.0036162856952716 -0.0026494029895859, 0.0035811761254146 -0.0026926144552672, 0.0035514680278433 -0.0026277972567452, 0.0035082562495578 -0.0026358994065604)), ((0.0035622709724147 -0.0025899872242740, 0.0036378915844144 -0.0026115929571147, 0.0036135849591288 -0.0025845857910639, 0.0035622709724147 -0.0025899872242740)), ((0.0035892783338432 -0.0058767593326593, 0.0035433658194148 -0.0059388758145762, 0.0035703731808433 -0.0059820872802576, 0.0036000812784146 -0.0058983650655000, 0.0035892783338432 -0.0058767593326593)), ((0.0035784753892718 -0.0022280911991930, 0.0035325628748434 -0.0022442954988235, 0.0035271614025577 -0.0022659012316641, 0.0035811761254146 -0.0022767040980845, 0.0035946798061289 -0.0022442954988235, 0.0035784753892718 -0.0022280911991930)), ((0.0035892783338432 -0.0024414478109945, 0.0035028547772721 -0.0025278707423571, 0.0035433658194148 -0.0025521771918029, 0.0035892783338432 -0.0024414478109945)), ((0.0035703731808433 -0.0023766306124725, 0.0035001540411292 -0.0024441485275996, 0.0034623437351294 -0.0024198420781538, 0.0034569422628437 -0.0025170678759368, 0.0034785481519865 -0.0025116664427266, 0.0035244606664149 -0.0024279442279691, 0.0035784753892718 -0.0023955356287081, 0.0035703731808433 -0.0023766306124725)), ((0.0035109569857006 -0.0021470697010405, 0.0035082562495578 -0.0021713761504863, 0.0035379643471291 -0.0021848797335117, 0.0035514680278433 -0.0021578725674609, 0.0035109569857006 -0.0021470697010405)), ((0.0035406650832719 -0.0060415030455694, 0.0034974533049864 -0.0061198238271168, 0.0035190591941292 -0.0061117216773015, 0.0035406650832719 -0.0060415030455694)), ((0.0035082562495578 -0.0023820320456827, 0.0034785481519865 -0.0023982363453132, 0.0035082562495578 -0.0023982363453132, 0.0035082562495578 -0.0023820320456827)), ((0.0034488400544152 -0.0026845123054519, 0.0034650444712722 -0.0027520302205790, 0.0034839496242722 -0.0027466287873688, 0.0034893510965579 -0.0026872130220570, 0.0034488400544152 -0.0026845123054519)), ((0.0034758474158436 -0.0006805805844808, 0.0034083290122725 -0.0007264927667672, 0.0034650444712722 -0.0007183906169519, 0.0034758474158436 -0.0006805805844808)), ((0.0034299349014152 -0.0025710822080385, 0.0034083290122725 -0.0026088922405096, 0.0034380371098438 -0.0026467022729808, 0.0034299349014152 -0.0025710822080385)), ((0.0034353363737009 -0.0059766858470474, 0.0034137304845582 -0.0059793865636525, 0.0034029275399868 -0.0060117951629135, 0.0034353363737009 -0.0059766858470474)), ((0.0033759201785583 -0.0023469227298166, 0.0033543142894155 -0.0023712291792624, 0.0033921245954154 -0.0024171413615488, 0.0033786209147011 -0.0024549513940199, 0.0033948253315582 -0.0024711556936504, 0.0034326356375581 -0.0024252435113640, 0.0034110297484153 -0.0024306449445742, 0.0034272341652724 -0.0024036377785234, 0.0033786209147011 -0.0023928349121030, 0.0033759201785583 -0.0023469227298166)), ((0.0033732194424155 -0.0042347236367692, 0.0033786209147011 -0.0042536286530048, 0.0034002268038439 -0.0042455265031896, 0.0033732194424155 -0.0042347236367692)), ((0.0033840223869868 -0.0025467757585927, 0.0033597157617012 -0.0025521771918029, 0.0033624164978441 -0.0025764836412486, 0.0033840223869868 -0.0025467757585927)), ((0.0033489128171298 -0.0054770532751071, 0.0033570150255584 -0.0055067611577631, 0.0033786209147011 -0.0054959582913427, 0.0033489128171298 -0.0054770532751071)), ((0.0033057010388443 -0.0020363403202321, 0.0033516135532727 -0.0020660482028881, 0.0033786209147011 -0.0020228367372067, 0.0033138032472728 -0.0020147345873915, 0.0033057010388443 -0.0020363403202321)), ((0.0033678179701298 -0.0020741503527033, 0.0033381098725584 -0.0020957560855440, 0.0033516135532727 -0.0021146611017795, 0.0033678179701298 -0.0020741503527033)), ((0.0033002995665586 -0.0072811319673022, 0.0032867958858444 -0.0072730298174870, 0.0032759929412730 -0.0073000369835378, 0.0033002995665586 -0.0072811319673022)), ((0.0032732922051301 -0.0021902811667218, 0.0032381826352731 -0.0022145876161676, 0.0032678907328444 -0.0022226897659828, 0.0032732922051301 -0.0021902811667218)), ((0.0032030730654161 -0.0052069816145989, 0.0032246789545589 -0.0052339887806497, 0.0032516863159874 -0.0052204851976243, 0.0032030730654161 -0.0052069816145989)), ((0.0032489855798445 -0.0060685102116202, 0.0031760657039876 -0.0060982180942761, 0.0031868686485590 -0.0061468309931676, 0.0032300804268446 -0.0061333274101422, 0.0032489855798445 -0.0060685102116202)), ((0.0032057738015590 -0.0069111337924059, 0.0031706642317020 -0.0069327395252466, 0.0031841679124162 -0.0069678488411126, 0.0032246789545589 -0.0069543452580872, 0.0032057738015590 -0.0069111337924059)), ((0.0032111752738447 -0.0017581665099087, 0.0030815399389880 -0.0017770715261442, 0.0030653355221309 -0.0018121808420103, 0.0030788392028452 -0.0018229837084306, 0.0031301531895593 -0.0017905751091696, 0.0031706642317020 -0.0018229837084306, 0.0032111752738447 -0.0017986772589849, 0.0032111752738447 -0.0017581665099087)), ((0.0031949708569876 -0.0064412091031216, 0.0031652627594163 -0.0064601141193571, 0.0031841679124162 -0.0064736177023826, 0.0031949708569876 -0.0064412091031216)), ((0.0031193502449879 -0.0009803601276449, 0.0030761384667023 -0.0010073672936957, 0.0031058465642736 -0.0010073672936957, 0.0031193502449879 -0.0009803601276449)), ((0.0030761384667023 -0.0005914569365130, 0.0030059193269883 -0.0006292669689842, 0.0030221237438454 -0.0006616755682452, 0.0030572333137024 -0.0006454712686147, 0.0030761384667023 -0.0005914569365130)), ((0.0030599340498452 -0.0018851001903475, 0.0030059193269883 -0.0018959030567679, 0.0029816127017027 -0.0019823259881305, 0.0030275252161311 -0.0019823259881305, 0.0030599340498452 -0.0018851001903475)), ((0.0030572333137024 -0.0001080286642033, 0.0030410288968453 -0.0000999265143880, 0.0029924156462741 -0.0001404372634643, 0.0030113207992740 -0.0001539408464897, 0.0030572333137024 -0.0001080286642033)), ((0.0030167222715597 -0.0073972627813207, 0.0029789119655598 -0.0074026642145309, 0.0030059193269883 -0.0074134670809512, 0.0030167222715597 -0.0073972627813207)), ((0.0029492038679885 -0.0074539778300275, 0.0029492038679885 -0.0074836857126834, 0.0029843134378455 -0.0074863864292885, 0.0029816127017027 -0.0074512771134224, 0.0029492038679885 -0.0074539778300275)), ((0.0029465031318457 -0.0015232041652665, 0.0029762112294170 -0.0015042991490309, 0.0029465031318457 -0.0014961969992157, 0.0029465031318457 -0.0015232041652665)), ((0.0029654082848456 -0.0052690980965158, 0.0029546053402742 -0.0053015066957768, 0.0029735104932741 -0.0052907038293565, 0.0029654082848456 -0.0052690980965158)), ((0.0029086928258458 -0.0076538308588036, 0.0028762839921317 -0.0077186480573256, 0.0029032913535601 -0.0077051444743001, 0.0029086928258458 -0.0076538308588036)), ((0.0028978898812744 -0.0004861289889148, 0.0028708825198460 -0.0004807275557047, 0.0028573788391317 -0.0005212383047809, 0.0028978898812744 -0.0005158368715707, 0.0028978898812744 -0.0004861289889148)), ((0.0028600795752746 -0.0018932023401628, 0.0028357729499890 -0.0019067059231882, 0.0028438751584175 -0.0019607202552898, 0.0028708825198460 -0.0019445159556593, 0.0028600795752746 -0.0018932023401628)), ((0.0028357729499890 -0.0077780638226374, 0.0028303714777033 -0.0078050709886882, 0.0028654810475603 -0.0077996695554780, 0.0028357729499890 -0.0077780638226374)), ((0.0028249700054176 -0.0017284586272527, 0.0028168677969890 -0.0017554657933036, 0.0028492766307032 -0.0017608672265137, 0.0028249700054176 -0.0017284586272527)), ((0.0028384736861318 -0.0021443689844354, 0.0028087655885605 -0.0021551718508558, 0.0028087655885605 -0.0021767775836964, 0.0028357729499890 -0.0021713761504863, 0.0028384736861318 -0.0021443689844354)), ((0.0028330722138461 -0.0015799192139732, 0.0027979626439891 -0.0015745177807631, 0.0027682545464178 -0.0016069263800240, 0.0028006633801320 -0.0016285321128647, 0.0028330722138461 -0.0015799192139732)), ((0.0028168677969890 -0.0012018188892617, 0.0027709552825607 -0.0012018188892617, 0.0027331449765608 -0.0012477310715481, 0.0027898604355606 -0.0012450303549430, 0.0028168677969890 -0.0012018188892617)), ((0.0028114663247034 -0.0029086717836738, 0.0027682545464178 -0.0029491825327500, 0.0028114663247034 -0.0029491825327500, 0.0028114663247034 -0.0029086717836738)), ((0.0028114663247034 -0.0076106193931223, 0.0027817582271320 -0.0076025172433070, 0.0027574516018464 -0.0076295244093578, 0.0028114663247034 -0.0076106193931223)), ((0.0027466486572750 -0.0021362668346202, 0.0027223420319894 -0.0021713761504863, 0.0027520501295607 -0.0021605732840659, 0.0027466486572750 -0.0021362668346202)), ((0.0027412471849893 -0.0006292669689842, 0.0027169405597037 -0.0006211648191690, 0.0027169405597037 -0.0006454712686147, 0.0026494221561326 -0.0006994856007163, 0.0027520501295607 -0.0006994856007163, 0.0027412471849893 -0.0006292669689842)), ((0.0027304442404180 -0.0074134670809512, 0.0026980354067038 -0.0074107663643462, 0.0026791302537039 -0.0074350728137919, 0.0027061376151323 -0.0074458756802122, 0.0027304442404180 -0.0074134670809512)), ((0.0027142398235609 -0.0047100497592637, 0.0026899331982753 -0.0047100497592637, 0.0026791302537039 -0.0047397576419196, 0.0027142398235609 -0.0047100497592637)), ((0.0026602251007039 -0.0001620429963049, 0.0026548236284182 -0.0002133566118015, 0.0026980354067038 -0.0002187580450117, 0.0027034368789895 -0.0001728458627253, 0.0026602251007039 -0.0001620429963049)), ((0.0026980354067038 -0.0060928166610659, 0.0026440206838469 -0.0060874152278558, 0.0026359184754183 -0.0061468309931676, 0.0026791302537039 -0.0061279259769320, 0.0026980354067038 -0.0060928166610659)), ((0.0026953346705609 -0.0027304244877383, 0.0026521228922754 -0.0027304244877383, 0.0026440206838469 -0.0027709352368146, 0.0026764295175610 -0.0027790373866298, 0.0026953346705609 -0.0027304244877383)), ((0.0026602251007039 -0.0077618595230069, 0.0026521228922754 -0.0077996695554780, 0.0026710280452753 -0.0078104724218984, 0.0026602251007039 -0.0077618595230069)), ((0.0026332177392755 -0.0063682897547843, 0.0026035096417042 -0.0063736911879945, 0.0025981081694185 -0.0063952969208352, 0.0026170133224184 -0.0063979976374403, 0.0026332177392755 -0.0063682897547843)), ((0.0026197140585612 -0.0073324455827988, 0.0025954074332756 -0.0073405477326140, 0.0026035096417042 -0.0073702556152699, 0.0026224147947041 -0.0073621534654547, 0.0026197140585612 -0.0073324455827988)), ((0.0026197140585612 -0.0016636414287308, 0.0025656993357043 -0.0016960500279918, 0.0025656993357043 -0.0017203564774375, 0.0026170133224184 -0.0016960500279918, 0.0026197140585612 -0.0016636414287308)), ((0.0026035096417042 -0.0019850267047356, 0.0025548963911329 -0.0019985302877610, 0.0025792030164185 -0.0020120338707864, 0.0026035096417042 -0.0019850267047356)), ((0.0025873052248471 -0.0039511483932356, 0.0025440934465615 -0.0039781555592864, 0.0025440934465615 -0.0039997612921271, 0.0025927066971328 -0.0039835569924966, 0.0025873052248471 -0.0039511483932356)), ((0.0025792030164185 -0.0014205769342734, 0.0025467941827044 -0.0014556862501394, 0.0025738015441328 -0.0014556862501394, 0.0025792030164185 -0.0014205769342734)), ((0.0025413927104187 -0.0088664526144856, 0.0024954801959903 -0.0088961604971415, 0.0024846772514189 -0.0089339705296126, 0.0025359912381330 -0.0089096640801669, 0.0025413927104187 -0.0088664526144856)), ((0.0025332905019901 -0.0024522506774148, 0.0024927794598474 -0.0024846592766758, 0.0025224875574188 -0.0024819585600707, 0.0025332905019901 -0.0024522506774148)), ((0.0024900787237046 -0.0074647806964478, 0.0025305897658473 -0.0074917878624986, 0.0025251882935616 -0.0074566785466326, 0.0024900787237046 -0.0074647806964478)), ((0.0025143853489902 -0.0015340070316868, 0.0024684728345618 -0.0015394084648970, 0.0024603706261333 -0.0015637149143427, 0.0025089838767045 -0.0015637149143427, 0.0025143853489902 -0.0015340070316868)), ((0.0024954801959903 -0.0063574868883640, 0.0024738743068475 -0.0063817933378098, 0.0024900787237046 -0.0064088005038606, 0.0025116846128474 -0.0063844940544148, 0.0024954801959903 -0.0063574868883640)), ((0.0024522684177047 -0.0060793130780405, 0.0023793485418479 -0.0061144223939066, 0.0024117573755620 -0.0061792395924286, 0.0024468669454191 -0.0061468309931676, 0.0024522684177047 -0.0060793130780405)), ((0.0024225603201334 -0.0024387470943894, 0.0024036551671335 -0.0024387470943894, 0.0023928522225621 -0.0024765571268606, 0.0023874507502764 -0.0024306449445742, 0.0023577426527051 -0.0024198420781538, 0.0023523411804194 -0.0024495499608098, 0.0023037279298482 -0.0024792578434657, 0.0023334360274195 -0.0025467757585927, 0.0023307352912766 -0.0025953886574842, 0.0024441662092762 -0.0025332721755673, 0.0024225603201334 -0.0024387470943894)), ((0.0023496404442766 -0.0088016354159636, 0.0023523411804194 -0.0088259418654093, 0.0024009544309907 -0.0088448468816449, 0.0024360640008477 -0.0088313432986195, 0.0024306625285620 -0.0088124382823839, 0.0023847500141336 -0.0088232411488043, 0.0023685455972765 -0.0087935332661484, 0.0023496404442766 -0.0088016354159636)), ((0.0024279617924191 -0.0058848614824746, 0.0024090566394192 -0.0059199707983407, 0.0024279617924191 -0.0059172700817356, 0.0024279617924191 -0.0058848614824746)), ((0.0023793485418479 -0.0012666360877837, 0.0023550419165623 -0.0012639353711786, 0.0023280345551338 -0.0012963439704396, 0.0022875235129911 -0.0012882418206243, 0.0022740198322769 -0.0013152489866751, 0.0023199323467053 -0.0013314532863056, 0.0023793485418479 -0.0012666360877837)), ((0.0023442389719909 -0.0069084330758008, 0.0022983264575625 -0.0069219366588262, 0.0023010271937053 -0.0069624474079025, 0.0023307352912766 -0.0069570459746923, 0.0023442389719909 -0.0069084330758008)), ((0.0023037279298482 -0.0072676283842768, 0.0023253338189910 -0.0072838326839073, 0.0023388374997052 -0.0072649276676717, 0.0023037279298482 -0.0072676283842768)), ((0.0023118301382767 -0.0047100497592637, 0.0022821220407054 -0.0047370569253146, 0.0023199323467053 -0.0047721662411806, 0.0023361367635623 -0.0047505605083400, 0.0023118301382767 -0.0047100497592637)), ((0.0022497132069913 -0.0087287160676264, 0.0022983264575625 -0.0087476210838620, 0.0023145308744196 -0.0087233146344162, 0.0022956257214196 -0.0087044096181806, 0.0022497132069913 -0.0087287160676264)), ((0.0022848227768483 -0.0060009922964931, 0.0022551146792769 -0.0060090944463084, 0.0022551146792769 -0.0060279994625440, 0.0022794213045626 -0.0060307001791490, 0.0022848227768483 -0.0060009922964931)), ((0.0022173043732771 -0.0069300388086415, 0.0022281073178485 -0.0069489438248771, 0.0022443117347056 -0.0069354402418517, 0.0022173043732771 -0.0069300388086415)), ((0.0022038006925629 -0.0052177844810192, 0.0021578881781345 -0.0051907773149684, 0.0021038734552775 -0.0052339887806497, 0.0021011727191347 -0.0052717988131209, 0.0021362822889917 -0.0052744995297260, 0.0022038006925629 -0.0052177844810192)), ((0.0021956984841343 -0.0066491642817129, 0.0021794940672772 -0.0066599671481332, 0.0021929977479915 -0.0066788721643688, 0.0021956984841343 -0.0066491642817129)), ((0.0021524867058488 -0.0039268419437898, 0.0021659903865630 -0.0039457469600254, 0.0021686911227059 -0.0039241412271848, 0.0021524867058488 -0.0039268419437898)), ((0.0021470852335631 -0.0056904098869087, 0.0020984719829918 -0.0057309206359849, 0.0021254793444203 -0.0057390227858001, 0.0021470852335631 -0.0056904098869087)), ((0.0021470852335631 -0.0012288260553125, 0.0021281800805632 -0.0012153224722871, 0.0020930705107061 -0.0012369282051277, 0.0021146763998489 -0.0012504317881532, 0.0021470852335631 -0.0012288260553125)), ((0.0021416837612774 -0.0078482824543695, 0.0021146763998489 -0.0078482824543695, 0.0020876690384205 -0.0078833917702356, 0.0021416837612774 -0.0078482824543695)), ((0.0021173771359918 -0.0060225980293338, 0.0020903697745633 -0.0060225980293338, 0.0020579609408491 -0.0060766123614354, 0.0020876690384205 -0.0060820137946456, 0.0021173771359918 -0.0060225980293338)), ((0.0020444572601349 -0.0013071468368599, 0.0020228513709921 -0.0013044461202548, 0.0020039462179922 -0.0013422561527260, 0.0020417565239921 -0.0013341540029107, 0.0020444572601349 -0.0013071468368599)), ((0.0020390557878492 -0.0080049240174643, 0.0020120484264207 -0.0080103254506745, 0.0020201506348493 -0.0080346319001202, 0.0020390557878492 -0.0080049240174643)), ((0.0019634351758495 -0.0071731033030989, 0.0019958440095637 -0.0071839061695192, 0.0020012454818494 -0.0072190154853853, 0.0020390557878492 -0.0071947090359396, 0.0020309535794207 -0.0071650011532837, 0.0019634351758495 -0.0071731033030989)), ((0.0020282528432778 -0.0073891606315055, 0.0019634351758495 -0.0074377735303970, 0.0020093476902779 -0.0074350728137919, 0.0020282528432778 -0.0073891606315055)), ((0.0019850410649923 -0.0071190889709973, 0.0019688366481352 -0.0070947825215515, 0.0019391285505639 -0.0071298918374176, 0.0019850410649923 -0.0071190889709973)), ((0.0019148219252783 -0.0013503583025412, 0.0019040189807069 -0.0013827669018022, 0.0019229241337068 -0.0013962704848276, 0.0019391285505639 -0.0013638618855666, 0.0019148219252783 -0.0013503583025412)), ((0.0019256248698497 -0.0066761714477637, 0.0018905152999926 -0.0066734707311587, 0.0018851138277070 -0.0067112807636298, 0.0019202233975640 -0.0067004778972095, 0.0019256248698497 -0.0066761714477637)), ((0.0019202233975640 -0.0080319311835151, 0.0018851138277070 -0.0080481354831456, 0.0018878145638498 -0.0080724419325914, 0.0019202233975640 -0.0080589383495659, 0.0019202233975640 -0.0080319311835151)), ((0.0019202233975640 -0.0068058058448077, 0.0018851138277070 -0.0068166087112280, 0.0019067197168497 -0.0068247108610433, 0.0019202233975640 -0.0068058058448077)), ((0.0018365005771357 -0.0040294691747830, 0.0018689094108499 -0.0040564763408338, 0.0019148219252783 -0.0040321698913881, 0.0018851138277070 -0.0039997612921271, 0.0018365005771357 -0.0040294691747830)), ((0.0019121211891354 -0.0046641375769773, 0.0018500042578499 -0.0046857433098180, 0.0018581064662785 -0.0047289547754993, 0.0018986175084212 -0.0047208526256841, 0.0019121211891354 -0.0046641375769773)), ((0.0019094204529926 -0.0071866068861243, 0.0018419020494214 -0.0072109133355701, 0.0019040189807069 -0.0072190154853853, 0.0019094204529926 -0.0071866068861243)), ((0.0018824130915641 -0.0078887932034457, 0.0018310991048500 -0.0078914939200508, 0.0018310991048500 -0.0079103989362864, 0.0018716101469927 -0.0079212018027067, 0.0018824130915641 -0.0078887932034457)), ((0.0018662086747070 -0.0008075142649196, 0.0018202961602786 -0.0007994121151044, 0.0017824858542788 -0.0008291199977603, 0.0018446027855643 -0.0008318207143654, 0.0018662086747070 -0.0008075142649196)), ((0.0018581064662785 -0.0079563111185728, 0.0018310991048500 -0.0079806175680185, 0.0018608072024213 -0.0079752161348084, 0.0018581064662785 -0.0079563111185728)), ((0.0018175954241358 -0.0066815728809739, 0.0018337998409929 -0.0067058793304196, 0.0018392013132786 -0.0066788721643688, 0.0018175954241358 -0.0066815728809739)), ((0.0018175954241358 -0.0082155799126607, 0.0017851865904216 -0.0082155799126607, 0.0017716829097074 -0.0082479885119217, 0.0017203689229933 -0.0082641928115522, 0.0017176681868505 -0.0082911999776030, 0.0018337998409929 -0.0082560906617370, 0.0018175954241358 -0.0082155799126607)), ((0.0018283983687072 -0.0081534634307438, 0.0018094932157072 -0.0081642662971642, 0.0018229968964215 -0.0081831713133997, 0.0018283983687072 -0.0081534634307438)), ((0.0017311718675647 -0.0008885357630721, 0.0017581792289932 -0.0009155429291229, 0.0017851865904216 -0.0008966379128874, 0.0017311718675647 -0.0008885357630721)), ((0.0016393468387079 -0.0075890136602816, 0.0016528505194221 -0.0076349258425680, 0.0016852593531363 -0.0076241229761477, 0.0016852593531363 -0.0076538308588036, 0.0017257703952790 -0.0076403272757782, 0.0017014637699934 -0.0075890136602816, 0.0016393468387079 -0.0075890136602816)), ((0.0016852593531363 -0.0012990446870446, 0.0017041645061362 -0.0013098475534650, 0.0017257703952790 -0.0012855411040192, 0.0016852593531363 -0.0012990446870446)), ((0.0016825586169935 -0.0001512401298846, 0.0017014637699934 -0.0001809480125405, 0.0017095659784219 -0.0001539408464897, 0.0016825586169935 -0.0001512401298846)), ((0.0016501497832793 -0.0010721844922177, 0.0016366461025651 -0.0010937902250584, 0.0016663542001364 -0.0011505052737651, 0.0016960622977077 -0.0011288995409245, 0.0016501497832793 -0.0010721844922177)), ((0.0016609527278507 -0.0064493112529368, 0.0016636534639935 -0.0064736177023826, 0.0016852593531363 -0.0064736177023826, 0.0016852593531363 -0.0064520119695419, 0.0016609527278507 -0.0064493112529368)), ((0.0016285438941365 -0.0071244904042074, 0.0016582519917078 -0.0071379939872328, 0.0016744564085649 -0.0071163882543922, 0.0016474490471364 -0.0071055853879718, 0.0016285438941365 -0.0071244904042074)), ((0.0016582519917078 -0.0083290100100742, 0.0016474490471364 -0.0083452143097047, 0.0016663542001364 -0.0083533164595199, 0.0016582519917078 -0.0083290100100742)), ((0.0016582519917078 -0.0043427523009725, 0.0016231424218508 -0.0043481537341827, 0.0016231424218508 -0.0044048687828894, 0.0016582519917078 -0.0043913651998640, 0.0016582519917078 -0.0043427523009725)), ((0.0016393468387079 -0.0008561271638111, 0.0016042372688509 -0.0008669300302315, 0.0016096387411366 -0.0009398493785687, 0.0016555512555650 -0.0008885357630721, 0.0016393468387079 -0.0008561271638111)), ((0.0016555512555650 -0.0070326660396346, 0.0016204416857080 -0.0070596732056854, 0.0016555512555650 -0.0070569724890804, 0.0016555512555650 -0.0070326660396346)), ((0.0016528505194221 -0.0003808010413166, 0.0016069380049938 -0.0003916039077370, 0.0016285438941365 -0.0004132096405776, 0.0016528505194221 -0.0003808010413166)), ((0.0015988357965652 -0.0076484294255934, 0.0015934343242795 -0.0076862394580646, 0.0016393468387079 -0.0077051444743001, 0.0016528505194221 -0.0076808380248544, 0.0015988357965652 -0.0076484294255934)), ((0.0016042372688509 -0.0085018558727995, 0.0016258431579937 -0.0085153594558249, 0.0016366461025651 -0.0084991551561944, 0.0016042372688509 -0.0085018558727995)), ((0.0016042372688509 -0.0070704760721058, 0.0015934343242795 -0.0070947825215515, 0.0016123394772794 -0.0071163882543922, 0.0016231424218508 -0.0070839796551312, 0.0016042372688509 -0.0070704760721058)), ((0.0016069380049938 -0.0043292487179471, 0.0015880328519938 -0.0043130444183166, 0.0015421203375654 -0.0043373508677624, 0.0016069380049938 -0.0043292487179471)), ((0.0015232151845655 -0.0011937167394464, 0.0015286166568512 -0.0012180231888922, 0.0015664269628511 -0.0012153224722871, 0.0015232151845655 -0.0011937167394464)), ((0.0015529232821368 -0.0007480984996078, 0.0015286166568512 -0.0007805070988688, 0.0015583247544225 -0.0007751056656587, 0.0015529232821368 -0.0007480984996078)), ((0.0015178137122798 -0.0072730298174870, 0.0015583247544225 -0.0072757305340920, 0.0015475218098511 -0.0072595262344615, 0.0015178137122798 -0.0072730298174870)), ((0.0015394196014226 -0.0071758040197040, 0.0015151129761370 -0.0071731033030989, 0.0014854048785657 -0.0072055119023599, 0.0014989085592799 -0.0072298183518056, 0.0015259159207084 -0.0072244169185955, 0.0015394196014226 -0.0071758040197040)), ((0.0015097115038513 -0.0032570642257294, 0.0014854048785657 -0.0032651663755446, 0.0014827041424228 -0.0032948742582006, 0.0015097115038513 -0.0032894728249904, 0.0015097115038513 -0.0032570642257294)), ((0.0015043100315656 -0.0070758775053159, 0.0014610982532800 -0.0070812789385261, 0.0014367916279944 -0.0071325925540227, 0.0014989085592799 -0.0071190889709973, 0.0015043100315656 -0.0070758775053159)), ((0.0014989085592799 -0.0004024067741573, 0.0014746019339943 -0.0004105089239725, 0.0014773026701371 -0.0004348153734183, 0.0015043100315656 -0.0004294139402081, 0.0014989085592799 -0.0004024067741573)), ((0.0014962078231370 -0.0011180966745041, 0.0014773026701371 -0.0011234981077143, 0.0014719011978514 -0.0011721110066058, 0.0015043100315656 -0.0011613081401854, 0.0014962078231370 -0.0011180966745041)), ((0.0014692004617086 -0.0004915304221250, 0.0013935798497089 -0.0005104354383606, 0.0014232879472802 -0.0005320411712012, 0.0014692004617086 -0.0004915304221250)), ((0.0014583975171372 -0.0005941576531181, 0.0013773754328518 -0.0005941576531181, 0.0013800761689946 -0.0006508727018249, 0.0013962805858517 -0.0006562741350350, 0.0014583975171372 -0.0005941576531181)), ((0.0014124850027088 -0.0032840713917802, 0.0014205872111373 -0.0033056771246209, 0.0014394923641373 -0.0032975749748056, 0.0014124850027088 -0.0032840713917802)), ((0.0014286894195659 -0.0085666730713214, 0.0013854776412803 -0.0085882788041621, 0.0014043827942803 -0.0085963809539773, 0.0014286894195659 -0.0085666730713214)), ((0.0014205872111373 -0.0004510196730488, 0.0013827769051375 -0.0004699246892843, 0.0013449665991376 -0.0004591218228640, 0.0013287621822805 -0.0004915304221250, 0.0012720467232808 -0.0005050340051504, 0.0012531415702808 -0.0005401433210165, 0.0012639445148522 -0.0005671504870673, 0.0013098570292806 -0.0005590483372521, 0.0013368643907091 -0.0005050340051504, 0.0014151857388516 -0.0004807275557047, 0.0014205872111373 -0.0004510196730488)), ((0.0014205872111373 -0.0061846410256387, 0.0013557695437090 -0.0061900424588489, 0.0013530688075662 -0.0062251517747150, 0.0014205872111373 -0.0061846410256387)), ((0.0013989813219946 -0.0007264927667672, 0.0013719739605661 -0.0007264927667672, 0.0013692732244232 -0.0007534999328180, 0.0013989813219946 -0.0007264927667672)), ((0.0013638717521376 -0.0013638618855666, 0.0013314629184234 -0.0013773654685920, 0.0013341636545662 -0.0014151755010632, 0.0013854776412803 -0.0013935697682225, 0.0013638717521376 -0.0013638618855666)), ((0.0013341636545662 -0.0035217344530275, 0.0013098570292806 -0.0035379387526580, 0.0013287621822805 -0.0035730480685240, 0.0013449665991376 -0.0035568437688935, 0.0013341636545662 -0.0035217344530275)), ((0.0013260614461377 -0.0003970053409471, 0.0012882511401378 -0.0003835017579217, 0.0012531415702808 -0.0004186110737878, 0.0012801489317093 -0.0004375160900234, 0.0013260614461377 -0.0003970053409471)), ((0.0013071562931378 -0.0009182436457280, 0.0012774481955665 -0.0009263457955433, 0.0012828496678522 -0.0009560536781992, 0.0013098570292806 -0.0009398493785687, 0.0013071562931378 -0.0009182436457280)), ((0.0012936526124235 -0.0033326842906717, 0.0012666452509951 -0.0033380857238819, 0.0012612437787094 -0.0033704943231429, 0.0012936526124235 -0.0033326842906717)), ((0.0012666452509951 -0.0027223223379231, 0.0012477400979951 -0.0027466287873688, 0.0012612437787094 -0.0027655338036044, 0.0012882511401378 -0.0027439280707637, 0.0012666452509951 -0.0027223223379231)), ((0.0012855504039950 -0.0038809297615034, 0.0012558423064237 -0.0038836304781085, 0.0012423386257095 -0.0039133383607644, 0.0012828496678522 -0.0039025354943441, 0.0012855504039950 -0.0038809297615034)), ((0.0012855504039950 -0.0080238290336999, 0.0012531415702808 -0.0080184276004897, 0.0012369371534238 -0.0080670404993812, 0.0012801489317093 -0.0080643397827761, 0.0012855504039950 -0.0080238290336999)), ((0.0012828496678522 -0.0008237185645501, 0.0012531415702808 -0.0008453242973908, 0.0012801489317093 -0.0008426235807857, 0.0012828496678522 -0.0008237185645501)), ((0.0012639445148522 -0.0010802866420330, 0.0011721194859954 -0.0010451773261669, 0.0010640900402816 -0.0010694837756126, 0.0010289804704246 -0.0011018923748736, 0.0009803672198533 -0.0010775859254279, 0.0009479583861392 -0.0010964909416635, 0.0010235789981389 -0.0011667095733956, 0.0010613893041387 -0.0011316002575295, 0.0010613893041387 -0.0010937902250584, 0.0010829951932815 -0.0010937902250584, 0.0011937253751382 -0.0011829138730261, 0.0012072290558524 -0.0011153959578990, 0.0012639445148522 -0.0010802866420330)), ((0.0012504408341380 -0.0032462613593091, 0.0012234334727095 -0.0032462613593091, 0.0011991268474239 -0.0032786699585701, 0.0012504408341380 -0.0032462613593091)), ((0.0011775209582811 -0.0004267132236030, 0.0011883239028525 -0.0004537203896538, 0.0012288349449952 -0.0004456182398386, 0.0011991268474239 -0.0004159103571827, 0.0011775209582811 -0.0004267132236030)), ((0.0011856231667097 -0.0007345949165824, 0.0011613165414241 -0.0007562006494231, 0.0011964261112811 -0.0007751056656587, 0.0012180320004238 -0.0007507992162129, 0.0011856231667097 -0.0007345949165824)), ((0.0011964261112811 -0.0079644132683881, 0.0011451121245670 -0.0079860190012287, 0.0011613165414241 -0.0080238290336999, 0.0012018275835668 -0.0080022233008592, 0.0011964261112811 -0.0079644132683881)), ((0.0011964261112811 -0.0009155429291229, 0.0011802216944240 -0.0009236450789382, 0.0011910246389954 -0.0009452508117788, 0.0011964261112811 -0.0009155429291229)), ((0.0011586158052812 -0.0057876356846916, 0.0011532143329955 -0.0058254457171628, 0.0011883239028525 -0.0058389493001882, 0.0011964261112811 -0.0058146428507424, 0.0011586158052812 -0.0057876356846916)), ((0.0011829224305668 -0.0027925409696552, 0.0011370099161384 -0.0028195481357060, 0.0011748202221383 -0.0028249495689162, 0.0011829224305668 -0.0027925409696552)), ((0.0011478128607098 -0.0075998165267019, 0.0011802216944240 -0.0076106193931223, 0.0011748202221383 -0.0075944150934918, 0.0011478128607098 -0.0075998165267019)), ((0.0011316084438527 -0.0079536104019677, 0.0011046010824243 -0.0079752161348084, 0.0011235062354242 -0.0079779168514135, 0.0011316084438527 -0.0079536104019677)), ((0.0011073018185671 -0.0078968953532610, 0.0011316084438527 -0.0079266032359169, 0.0011316084438527 -0.0078995960698661, 0.0011073018185671 -0.0078968953532610)), ((0.0010316812065674 -0.0042320229201642, 0.0010208782619960 -0.0042860372522658, 0.0010613893041387 -0.0042941394020810, 0.0010748929848530 -0.0042563293696099, 0.0010316812065674 -0.0042320229201642)), ((0.0010505863595673 -0.0008561271638111, 0.0009992723728533 -0.0008696307468365, 0.0009965716367104 -0.0008912364796772, 0.0010235789981389 -0.0009047400627026, 0.0010505863595673 -0.0008561271638111)), ((0.0009911701644247 -0.0048558884559382, 0.0009695642752819 -0.0049045013548297, 0.0010397834149960 -0.0049288078042754, 0.0010451848872816 -0.0048882970551992, 0.0009911701644247 -0.0048558884559382)), ((0.0009857686921390 -0.0004402168066284, 0.0010181775258532 -0.0004456182398386, 0.0010424841511388 -0.0004240125069979, 0.0010073745812818 -0.0004105089239725, 0.0009857686921390 -0.0004402168066284)), ((0.0010289804704246 -0.0015529120479224, 0.0010127760535675 -0.0015583134811326, 0.0010181775258532 -0.0015853206471834, 0.0010289804704246 -0.0015529120479224)), ((0.0010181775258532 -0.0041077899563304, 0.0009803672198533 -0.0040915856566999, 0.0009749657475676 -0.0041347971223812, 0.0010181775258532 -0.0041077899563304)), ((0.0010073745812818 -0.0057012127533290, 0.0009803672198533 -0.0056769063038832, 0.0008858414548537 -0.0056985120367239, 0.0008804399825680 -0.0057147163363544, 0.0009209510247107 -0.0057363220691951, 0.0008831407187109 -0.0057525263688256, 0.0008831407187109 -0.0057714313850611, 0.0009182502885679 -0.0057714313850611, 0.0009290532331392 -0.0057444242190103, 0.0009587613307106 -0.0057525263688256, 0.0009452576499963 -0.0057741321016662, 0.0009641628029962 -0.0057822342514815, 0.0010073745812818 -0.0057012127533290)), ((0.0009614620668534 -0.0050206321688482, 0.0009722650114248 -0.0050422379016889, 0.0010073745812818 -0.0050260336020584, 0.0009803672198533 -0.0050071285858228, 0.0009614620668534 -0.0050206321688482)), ((0.0009803672198533 -0.0073270441495886, 0.0009830679559962 -0.0073810584816903, 0.0010046738451389 -0.0073837591982953, 0.0010019731089961 -0.0073270441495886, 0.0009803672198533 -0.0073270441495886)), ((0.0009128488162822 -0.0039376448102102, 0.0009101480801393 -0.0039700534094712, 0.0009452576499963 -0.0039889584257067, 0.0009911701644247 -0.0039646519762610, 0.0009641628029962 -0.0039295426603949, 0.0009128488162822 -0.0039376448102102)), ((0.0009398561777106 -0.0043886644832589, 0.0009857686921390 -0.0043913651998640, 0.0009695642752819 -0.0043751609002335, 0.0009398561777106 -0.0043886644832589)), ((0.0009101480801393 -0.0001350358302541, 0.0009641628029962 -0.0001566415630948, 0.0009641628029962 -0.0001215322472287, 0.0009101480801393 -0.0001350358302541)), ((0.0009479583861392 -0.0041537021386168, 0.0008804399825680 -0.0041780085880625, 0.0008750385102823 -0.0041969136042981, 0.0009128488162822 -0.0042158186205337, 0.0009479583861392 -0.0041537021386168)), ((0.0009236517608536 -0.0007994121151044, 0.0009317539692821 -0.0008318207143654, 0.0009452576499963 -0.0008129156981298, 0.0009236517608536 -0.0007994121151044)), ((0.0009317539692821 -0.0003808010413166, 0.0008696370379966 -0.0003970053409471, 0.0008750385102823 -0.0004186110737878, 0.0009155495524250 -0.0004240125069979, 0.0009317539692821 -0.0003808010413166)), ((0.0009101480801393 -0.0009803601276449, 0.0008831407187109 -0.0009938637106703, 0.0009047466078536 -0.0010208708767212, 0.0009263524969964 -0.0010046665770907, 0.0009101480801393 -0.0009803601276449)), ((0.0009209510247107 -0.0013422561527260, 0.0009020458717108 -0.0013557597357514, 0.0009155495524250 -0.0013773654685920, 0.0009209510247107 -0.0013422561527260)), ((0.0009128488162822 -0.0040159655917576, 0.0008777392464252 -0.0040240677415728, 0.0008993451355679 -0.0040591770574389, 0.0009209510247107 -0.0040402720412033, 0.0009128488162822 -0.0040159655917576)), ((0.0008696370379966 -0.0002646702272981, 0.0008804399825680 -0.0002862759601387, 0.0009155495524250 -0.0002835752435337, 0.0008993451355679 -0.0002484659276676, 0.0008696370379966 -0.0002646702272981)), ((0.0009101480801393 -0.0067301857798654, 0.0008750385102823 -0.0067382879296806, 0.0008723377741395 -0.0067842001119670, 0.0009101480801393 -0.0067571929459162, 0.0009101480801393 -0.0067301857798654)), ((0.0009101480801393 -0.0066518649983180, 0.0008561333572824 -0.0066815728809739, 0.0008723377741395 -0.0067004778972095, 0.0009074473439965 -0.0066923757473942, 0.0009101480801393 -0.0066518649983180)), ((0.0008966443994251 -0.0050719457843448, 0.0008561333572824 -0.0050827486507651, 0.0008507318849967 -0.0051205586832363, 0.0008750385102823 -0.0051394636994718, 0.0009074473439965 -0.0051043543836058, 0.0008966443994251 -0.0050719457843448)), ((0.0009020458717108 -0.0040888849400948, 0.0008399289404253 -0.0041158921061456, 0.0008966443994251 -0.0041131913895405, 0.0009020458717108 -0.0040888849400948)), ((0.0009020458717108 -0.0047208526256841, 0.0008669363018538 -0.0047316554921044, 0.0008831407187109 -0.0047478597917349, 0.0009020458717108 -0.0047208526256841)), ((0.0008912429271394 -0.0012018188892617, 0.0008507318849967 -0.0012072203224718, 0.0008777392464252 -0.0012207239054973, 0.0008912429271394 -0.0012018188892617)), ((0.0008750385102823 -0.0003132831261896, 0.0008507318849967 -0.0003483924420556, 0.0007886149537112 -0.0003456917254505, 0.0007724105368541 -0.0003889031911319, 0.0008480311488538 -0.0003672974582912, 0.0008750385102823 -0.0003132831261896)), ((0.0008048193705683 -0.0058740586160543, 0.0008075201067111 -0.0059010657821051, 0.0008750385102823 -0.0058956643488949, 0.0008372282042825 -0.0058605550330288, 0.0008048193705683 -0.0058740586160543)), ((0.0008507318849967 -0.0045237003135131, 0.0008129215789968 -0.0045372038965385, 0.0008318267319968 -0.0046263275445062, 0.0008669363018538 -0.0045993203784554, 0.0008507318849967 -0.0045237003135131)), ((0.0008615348295681 -0.0084910530063791, 0.0008129215789968 -0.0084991551561944, 0.0008345274681396 -0.0085099580226147, 0.0008615348295681 -0.0084910530063791)), ((0.0008615348295681 -0.0036891788825426, 0.0007778120091398 -0.0037053831821731, 0.0008075201067111 -0.0037729010973001, 0.0008588340934252 -0.0037269889150137, 0.0008615348295681 -0.0036891788825426)), ((0.0008561333572824 -0.0007832078154739, 0.0007913156898541 -0.0008183171313400, 0.0008048193705683 -0.0008615285970213, 0.0008318267319968 -0.0008696307468365, 0.0008507318849967 -0.0008480250139959, 0.0008561333572824 -0.0007832078154739)), ((0.0008291259958539 -0.0057309206359849, 0.0008237245235682 -0.0057471249356154, 0.0008507318849967 -0.0057525263688256, 0.0008291259958539 -0.0057309206359849)), ((0.0007886149537112 -0.0056174905385714, 0.0008075201067111 -0.0056823077370934, 0.0008453304127110 -0.0056634027208578, 0.0008318267319968 -0.0056201912551765, 0.0007886149537112 -0.0056174905385714)), ((0.0006292715212833 -0.0050287343186635, 0.0007400017031400 -0.0050638436345295, 0.0008183230512825 -0.0050476393348990, 0.0008426296765682 -0.0050098293024279, 0.0007021913971401 -0.0050233328854533, 0.0006616803549974 -0.0049693185533517, 0.0006292715212833 -0.0050287343186635)), ((0.0007967171621398 -0.0060766123614354, 0.0008048193705683 -0.0061090209606964, 0.0008318267319968 -0.0061171231105117, 0.0008345274681396 -0.0060982180942761, 0.0007967171621398 -0.0060766123614354)), ((0.0008345274681396 -0.0065492377673249, 0.0007859142175684 -0.0065735442167706, 0.0007886149537112 -0.0066194563990570, 0.0008345274681396 -0.0065492377673249)), ((0.0007805127452827 -0.0068463165938839, 0.0007670090645684 -0.0068787251931449, 0.0007886149537112 -0.0068868273429602, 0.0007886149537112 -0.0069192359422212, 0.0008129215789968 -0.0069219366588262, 0.0008318267319968 -0.0069057323591957, 0.0008237245235682 -0.0068760244765398, 0.0007805127452827 -0.0068463165938839)), ((0.0007967171621398 -0.0051826751651532, 0.0007832134814255 -0.0052150837644142, 0.0008021186344255 -0.0052366894972548, 0.0008210237874254 -0.0052015801813887, 0.0007967171621398 -0.0051826751651532)), ((0.0007373009669971 -0.0047613633747603, 0.0007427024392828 -0.0048018741238365, 0.0007156950778544 -0.0048558884559382, 0.0007670090645684 -0.0048639906057534, 0.0008021186344255 -0.0048450855895179, 0.0008102208428540 -0.0048099762736518, 0.0007373009669971 -0.0047613633747603)), ((0.0007643083284256 -0.0064142019370707, 0.0007940164259969 -0.0064331069533063, 0.0008102208428540 -0.0064006983540453, 0.0007643083284256 -0.0064142019370707)), ((0.0007589068561399 -0.0000351093158661, 0.0006832862441402 -0.0000648171985220, 0.0006589796188546 -0.0001377365468592, 0.0006751840357117 -0.0001593422796999, 0.0007373009669971 -0.0001701451461202, 0.0008021186344255 -0.0001161308140185, 0.0008075201067111 -0.0000945250811779, 0.0007589068561399 -0.0000351093158661)), ((0.0006562788827117 -0.0041672057216422, 0.0006238700489976 -0.0042536286530048, 0.0006670818272831 -0.0042374243533743, 0.0007129943417115 -0.0042536286530048, 0.0006940891887116 -0.0043130444183166, 0.0007400017031400 -0.0043670587504183, 0.0006913884525687 -0.0044021680662843, 0.0007183958139972 -0.0044183723659148, 0.0007670090645684 -0.0043670587504183, 0.0007318994947114 -0.0043130444183166, 0.0007859142175684 -0.0042590300862150, 0.0007670090645684 -0.0042185193371387, 0.0006724832995688 -0.0042212200537438, 0.0006778847718545 -0.0041888114544828, 0.0006562788827117 -0.0041672057216422)), ((0.0007805127452827 -0.0082479885119217, 0.0007589068561399 -0.0082803971111827, 0.0007832134814255 -0.0082722949613675, 0.0007805127452827 -0.0082479885119217)), ((0.0007778120091398 -0.0056796070204883, 0.0007508046477114 -0.0056931106035137, 0.0007805127452827 -0.0056958113201188, 0.0007778120091398 -0.0056796070204883)), ((0.0007210965501401 -0.0039241412271848, 0.0007535053838542 -0.0039565498264457, 0.0007805127452827 -0.0039403455268152, 0.0007427024392828 -0.0039106376441593, 0.0007210965501401 -0.0039241412271848)), ((0.0007129943417115 -0.0007913099652891, 0.0007156950778544 -0.0008399228641806, 0.0007562061199971 -0.0008480250139959, 0.0007751112729970 -0.0007967113984993, 0.0007129943417115 -0.0007913099652891)), ((0.0007508046477114 -0.0036270624006257, 0.0007156950778544 -0.0036648724330968, 0.0007454031754257 -0.0036891788825426, 0.0007697098007113 -0.0036648724330968, 0.0007508046477114 -0.0036270624006257)), ((0.0007562061199971 -0.0051907773149684, 0.0007318994947114 -0.0052015801813887, 0.0007427024392828 -0.0052204851976243, 0.0007562061199971 -0.0051907773149684)), ((0.0007102936055687 -0.0068922287761703, 0.0007318994947114 -0.0069192359422212, 0.0007373009669971 -0.0068976302093805, 0.0007102936055687 -0.0068922287761703)), ((0.0007210965501401 -0.0060550066285948, 0.0006454759381403 -0.0060928166610659, 0.0006562788827117 -0.0061306266935371, 0.0006994906609973 -0.0061360281267473, 0.0007318994947114 -0.0060874152278558, 0.0007210965501401 -0.0060550066285948)), ((0.0006994906609973 -0.0056985120367239, 0.0006859869802830 -0.0057147163363544, 0.0007102936055687 -0.0057363220691951, 0.0007210965501401 -0.0057120156197493, 0.0006994906609973 -0.0056985120367239)), ((0.0006481766742832 -0.0053420174448530, 0.0006778847718545 -0.0053744260441140, 0.0007075928694258 -0.0053636231776937, 0.0006940891887116 -0.0053420174448530, 0.0006481766742832 -0.0053420174448530)), ((0.0005590523815693 -0.0054959582913427, 0.0005644538538549 -0.0055256661739986, 0.0006022641598548 -0.0055499726234444, 0.0006103663682833 -0.0056120891053613, 0.0006454759381403 -0.0056417969880172, 0.0006886877164259 -0.0056282934049918, 0.0006994906609973 -0.0055607754898647, 0.0006265707851404 -0.0055607754898647, 0.0006076656321405 -0.0055175640241834, 0.0005590523815693 -0.0054959582913427)), ((0.0006994906609973 -0.0002079551785913, 0.0006238700489976 -0.0002349623446422, 0.0006373737297118 -0.0002808745269286, 0.0006994906609973 -0.0002646702272981, 0.0006994906609973 -0.0002079551785913)), ((0.0006967899248544 -0.0029572846825652, 0.0006454759381403 -0.0029761896988008, 0.0006481766742832 -0.0029923939984313, 0.0006751840357117 -0.0029950947150364, 0.0006967899248544 -0.0029572846825652)), ((0.0006481766742832 -0.0048855963385941, 0.0006373737297118 -0.0049450121039059, 0.0006589796188546 -0.0049504135371161, 0.0006913884525687 -0.0049099027880398, 0.0006481766742832 -0.0048855963385941)), ((0.0006778847718545 -0.0004132096405776, 0.0006535781465689 -0.0004240125069979, 0.0006643810911403 -0.0004483189564437, 0.0006778847718545 -0.0004132096405776)), ((0.0006373737297118 -0.0044129709327047, 0.0005941619514263 -0.0044993938640673, 0.0006454759381403 -0.0045345031799334, 0.0006751840357117 -0.0044831895644368, 0.0006373737297118 -0.0044129709327047)), ((0.0006427752019975 -0.0006292669689842, 0.0006454759381403 -0.0006535734184299, 0.0006724832995688 -0.0006643762848503, 0.0006751840357117 -0.0006346684021944, 0.0006427752019975 -0.0006292669689842)), ((0.0006751840357117 -0.0065654420669554, 0.0006481766742832 -0.0065600406337452, 0.0005644538538549 -0.0066086535326367, 0.0004672273527125 -0.0065789456499808, 0.0004456214635697 -0.0066356606986875, 0.0005212420755694 -0.0066896750307892, 0.0006319722574261 -0.0066869743141841, 0.0006751840357117 -0.0065654420669554)), ((0.0006265707851404 -0.0000162042996305, 0.0006508774104260 -0.0000243064494457, 0.0006589796188546 -0.0000027007166051, 0.0006265707851404 -0.0000162042996305)), ((0.0006400744658547 -0.0070029581569787, 0.0006157678405690 -0.0070542717724753, 0.0006346729935690 -0.0070623739222905, 0.0006562788827117 -0.0070488703392651, 0.0006400744658547 -0.0070029581569787)), ((0.0006508774104260 -0.0069543452580872, 0.0006292715212833 -0.0069462431082720, 0.0006157678405690 -0.0069813524241381, 0.0006454759381403 -0.0069759509909279, 0.0006508774104260 -0.0069543452580872)), ((0.0006184685767119 -0.0022821055312946, 0.0005995634237120 -0.0022983098309251, 0.0006022641598548 -0.0023496234464217, 0.0006265707851404 -0.0023712291792624, 0.0006481766742832 -0.0023172148471607, 0.0006184685767119 -0.0022821055312946)), ((0.0005779575345692 -0.0047721662411806, 0.0006022641598548 -0.0048126769902569, 0.0006454759381403 -0.0047802683909959, 0.0005968626875691 -0.0047586626581552, 0.0005779575345692 -0.0047721662411806)), ((0.0006454759381403 -0.0053987324935598, 0.0005995634237120 -0.0054095353599801, 0.0006292715212833 -0.0054311410928207, 0.0006454759381403 -0.0053987324935598)), ((0.0006427752019975 -0.0036405659836511, 0.0005941619514263 -0.0036729745829121, 0.0006400744658547 -0.0036837774493324, 0.0006427752019975 -0.0036405659836511)), ((0.0006373737297118 -0.0045993203784554, 0.0005914612152834 -0.0046155246780859, 0.0005914612152834 -0.0046344296943214, 0.0006373737297118 -0.0046317289777163, 0.0006373737297118 -0.0045993203784554)), ((0.0005617531177121 -0.0040537756242287, 0.0005833590068549 -0.0040888849400948, 0.0006346729935690 -0.0040753813570694, 0.0005914612152834 -0.0040402720412033, 0.0005617531177121 -0.0040537756242287)), ((0.0006292715212833 -0.0029329782331195, 0.0005833590068549 -0.0029383796663297, 0.0005671545899978 -0.0029653868323805, 0.0006319722574261 -0.0029653868323805, 0.0006292715212833 -0.0029329782331195)), ((0.0005887604791406 -0.0008156164147349, 0.0005644538538549 -0.0008561271638111, 0.0006103663682833 -0.0008723314634416, 0.0006319722574261 -0.0008561271638111, 0.0006292715212833 -0.0008237185645501, 0.0005887604791406 -0.0008156164147349)), ((0.0005023369225695 -0.0042131179039286, 0.0005293442839979 -0.0043238472847370, 0.0005725560622835 -0.0043643580338132, 0.0006292715212833 -0.0043562558839979, 0.0005617531177121 -0.0042752343858455, 0.0005833590068549 -0.0042212200537438, 0.0005536509092836 -0.0041942128876930, 0.0005023369225695 -0.0042131179039286)), ((0.0005671545899978 -0.0018688958907170, 0.0005833590068549 -0.0019202095062136, 0.0006211693128547 -0.0019256109394238, 0.0006265707851404 -0.0018851001903475, 0.0005671545899978 -0.0018688958907170)), ((0.0006130671044262 -0.0064952234352232, 0.0005590523815693 -0.0065087270182486, 0.0005509501731407 -0.0065438363341147, 0.0005968626875691 -0.0065465370507198, 0.0006130671044262 -0.0064952234352232)), ((0.0005482494369979 -0.0041239942559609, 0.0005644538538549 -0.0041591035718269, 0.0006076656321405 -0.0041347971223812, 0.0005806582707120 -0.0041158921061456, 0.0005482494369979 -0.0041239942559609)), ((0.0005806582707120 -0.0016069263800240, 0.0005698553261406 -0.0016420356958901, 0.0005914612152834 -0.0016528385623104, 0.0006076656321405 -0.0016312328294698, 0.0005806582707120 -0.0016069263800240)), ((0.0005617531177121 -0.0038296161460069, 0.0006076656321405 -0.0038350175792170, 0.0005995634237120 -0.0038188132795865, 0.0005617531177121 -0.0038296161460069)), ((0.0005914612152834 -0.0070218631732143, 0.0005860597429977 -0.0069921552905584, 0.0005374464924265 -0.0070110603067940, 0.0005671545899978 -0.0070353667562397, 0.0005914612152834 -0.0070218631732143)), ((0.0005617531177121 -0.0030085982980618, 0.0005752567984263 -0.0030356054641126, 0.0005887604791406 -0.0030112990146669, 0.0005617531177121 -0.0030085982980618)), ((0.0005806582707120 -0.0035757487851291, 0.0005671545899978 -0.0035865516515495, 0.0005860597429977 -0.0036027559511799, 0.0005806582707120 -0.0035757487851291)), ((0.0005860597429977 -0.0056931106035137, 0.0005239428117122 -0.0056525998544375, 0.0004942347141409 -0.0056796070204883, 0.0004159133659984 -0.0056823077370934, 0.0004780302972839 -0.0057309206359849, 0.0005698553261406 -0.0057309206359849, 0.0005860597429977 -0.0056931106035137)), ((0.0005455487008550 -0.0026521037061909, 0.0005752567984263 -0.0026791108722418, 0.0005725560622835 -0.0026467022729808, 0.0005455487008550 -0.0026521037061909)), ((0.0005590523815693 -0.0053798274773242, 0.0005320450201408 -0.0053906303437445, 0.0005428479647122 -0.0054041339267699, 0.0005698553261406 -0.0053960317769547, 0.0005590523815693 -0.0053798274773242)), ((0.0004321177828555 -0.0007291934833723, 0.0004402199912840 -0.0007751056656587, 0.0004969354502838 -0.0007967113984993, 0.0005536509092836 -0.0007697042324485, 0.0005320450201408 -0.0007291934833723, 0.0004915339779981 -0.0007562006494231, 0.0004321177828555 -0.0007291934833723)), ((0.0005212420755694 -0.0015718170641580, 0.0005347457562836 -0.0016069263800240, 0.0005509501731407 -0.0015880213637885, 0.0005212420755694 -0.0015718170641580)), ((0.0005401472285693 -0.0038998347777390, 0.0004699280888553 -0.0039052362109492, 0.0004429207274268 -0.0039403455268152, 0.0004699280888553 -0.0039619512596559, 0.0005266435478551 -0.0039376448102102, 0.0005401472285693 -0.0038998347777390)), ((0.0004834317695695 -0.0035055301533970, 0.0005077383948552 -0.0035298366028427, 0.0004969354502838 -0.0035649459187088, 0.0005293442839979 -0.0035568437688935, 0.0005374464924265 -0.0035379387526580, 0.0004834317695695 -0.0035055301533970)), ((0.0005131398671409 -0.0040456734744135, 0.0004915339779981 -0.0040672792072541, 0.0005104391309980 -0.0040942863733049, 0.0005320450201408 -0.0040699799238592, 0.0005131398671409 -0.0040456734744135)), ((0.0005266435478551 -0.0022848062478997, 0.0004942347141409 -0.0023280177135810, 0.0005293442839979 -0.0023172148471607, 0.0005266435478551 -0.0022848062478997)), ((0.0004645266165696 -0.0006535734184299, 0.0004753295611410 -0.0007021863173214, 0.0005212420755694 -0.0006940841675062, 0.0004969354502838 -0.0006427705520096, 0.0004645266165696 -0.0006535734184299)), ((0.0005185413394266 -0.0025116664427266, 0.0004915339779981 -0.0025197685925419, 0.0004942347141409 -0.0025386736087775, 0.0005185413394266 -0.0025116664427266)), ((0.0004348185189983 -0.0028492560183620, 0.0004510229358554 -0.0028816646176229, 0.0004969354502838 -0.0028573581681772, 0.0004699280888553 -0.0028357524353365, 0.0004348185189983 -0.0028492560183620)), ((0.0004942347141409 -0.0067923022617823, 0.0004510229358554 -0.0068004044115975, 0.0004726288249982 -0.0068166087112280, 0.0004942347141409 -0.0067923022617823)), ((0.0004780302972839 -0.0008534264472060, 0.0004564244081411 -0.0009047400627026, 0.0004915339779981 -0.0008912364796772, 0.0004780302972839 -0.0008534264472060)), ((0.0004267163105698 -0.0023604263128420, 0.0003916067407128 -0.0024360463777843, 0.0004375192551412 -0.0024549513940199, 0.0004780302972839 -0.0024252435113640, 0.0004807310334267 -0.0023847327622878, 0.0004267163105698 -0.0023604263128420)), ((0.0004159133659984 -0.0022496969320337, 0.0004537236719982 -0.0022550983652438, 0.0004780302972839 -0.0022226897659828, 0.0004294170467126 -0.0022199890493777, 0.0004159133659984 -0.0022496969320337)), ((0.0004429207274268 -0.0061090209606964, 0.0004780302972839 -0.0060982180942761, 0.0004564244081411 -0.0060874152278558, 0.0004429207274268 -0.0061090209606964)), ((0.0004537236719982 -0.0027709352368146, 0.0004564244081411 -0.0028195481357060, 0.0004726288249982 -0.0027979424028654, 0.0004537236719982 -0.0027709352368146)), ((0.0004294170467126 -0.0018526915910865, 0.0004240155744269 -0.0018823994737424, 0.0004510229358554 -0.0018878009069526, 0.0004672273527125 -0.0018634944575069, 0.0004294170467126 -0.0018526915910865)), ((0.0004618258804268 -0.0043022415518963, 0.0004132126298555 -0.0042887379688709, 0.0003835045322842 -0.0043157451349217, 0.0003943074768556 -0.0043508544507878, 0.0003564971708558 -0.0043886644832589, 0.0003645993792843 -0.0044210730825199, 0.0004321177828555 -0.0043940659164691, 0.0004618258804268 -0.0043022415518963)), ((0.0003808037961414 -0.0003348888590302, 0.0004213148382841 -0.0004213117903929, 0.0004564244081411 -0.0004267132236030, 0.0004618258804268 -0.0003970053409471, 0.0004294170467126 -0.0003375895756353, 0.0003808037961414 -0.0003348888590302)), ((0.0004591251442839 -0.0037404924980392, 0.0004213148382841 -0.0037323903482239, 0.0003591979069986 -0.0037756018139052, 0.0003889060045699 -0.0037945068301408, 0.0004591251442839 -0.0037404924980392)), ((0.0004510229358554 -0.0036135588176003, 0.0004051104214270 -0.0036243616840206, 0.0003700008515700 -0.0036648724330968, 0.0004186141021412 -0.0036675731497019, 0.0004510229358554 -0.0036135588176003)), ((0.0004483221997125 -0.0044588831149911, 0.0004105118937127 -0.0044669852648063, 0.0003970082129985 -0.0045020945806724, 0.0004375192551412 -0.0044939924308571, 0.0004483221997125 -0.0044588831149911)), ((0.0004429207274268 -0.0005455447542266, 0.0004159133659984 -0.0005374426044114, 0.0003835045322842 -0.0005617490538571, 0.0004294170467126 -0.0005644497704622, 0.0004429207274268 -0.0005455447542266)), ((0.0003537964347129 -0.0025764836412486, 0.0003186868648559 -0.0026629065726113, 0.0003997089491413 -0.0026845123054519, 0.0004375192551412 -0.0026358994065604, 0.0004078111575698 -0.0026196951069299, 0.0004024096852841 -0.0025818850744588, 0.0003537964347129 -0.0025764836412486)), ((0.0003943074768556 -0.0068544187436992, 0.0003781030599985 -0.0068841266263551, 0.0004024096852841 -0.0069003309259856, 0.0004240155744269 -0.0068679223267246, 0.0003943074768556 -0.0068544187436992)), ((0.0002970809757131 -0.0060847145112507, 0.0003348912817130 -0.0061549331429828, 0.0004078111575698 -0.0061522324263778, 0.0004078111575698 -0.0060874152278558, 0.0002970809757131 -0.0060847145112507)), ((0.0003618986431414 -0.0032651663755446, 0.0003835045322842 -0.0033029764080158, 0.0003943074768556 -0.0032786699585701, 0.0003618986431414 -0.0032651663755446)), ((0.0003808037961414 -0.0065951499496113, 0.0003618986431414 -0.0066194563990570, 0.0003862052684271 -0.0066302592654773, 0.0003808037961414 -0.0065951499496113)), ((0.0003564971708558 -0.0038296161460069, 0.0003618986431414 -0.0038539225954526, 0.0003835045322842 -0.0038512218788475, 0.0003754023238557 -0.0038269154294018, 0.0003564971708558 -0.0038296161460069)), ((0.0003754023238557 -0.0045912182286401, 0.0003348912817130 -0.0046020210950604, 0.0003375920178558 -0.0046479332773468, 0.0003645993792843 -0.0046371304109265, 0.0003754023238557 -0.0045912182286401)), ((0.0002808765588561 -0.0039592505430508, 0.0003186868648559 -0.0040078634419423, 0.0003645993792843 -0.0039862577091017, 0.0003375920178558 -0.0039484476766305, 0.0002808765588561 -0.0039592505430508)), ((0.0002727743504275 -0.0040834835068846, 0.0003267890732844 -0.0041185928227507, 0.0003564971708558 -0.0040780820736745, 0.0002727743504275 -0.0040834835068846)), ((0.0003132853925702 -0.0044723866980165, 0.0003402927539987 -0.0044777881312267, 0.0003537964347129 -0.0044561823983860, 0.0003213876009987 -0.0044507809651758, 0.0003132853925702 -0.0044723866980165)), ((0.0003213876009987 -0.0054905568581326, 0.0003024824479988 -0.0055094618743681, 0.0003321905455701 -0.0055337683238139, 0.0003456942262844 -0.0054959582913427, 0.0003213876009987 -0.0054905568581326)), ((0.0003375920178558 -0.0032219549098633, 0.0002808765588561 -0.0032381592094938, 0.0002808765588561 -0.0032867721083853, 0.0003186868648559 -0.0032705678087548, 0.0003375920178558 -0.0032219549098633)), ((0.0003321905455701 -0.0031463348449210, 0.0002862780311417 -0.0031436341283159, 0.0002700736142847 -0.0031760427275769, 0.0003240883371416 -0.0031814441607871, 0.0003321905455701 -0.0031463348449210)), ((0.0003294898094273 -0.0066059528160316, 0.0002754750865704 -0.0066329599820824, 0.0003078839202845 -0.0066518649983180, 0.0003294898094273 -0.0066059528160316)), ((0.0002673728781418 -0.0005833547866978, 0.0002592706697133 -0.0006049605195385, 0.0003078839202845 -0.0006319676855893, 0.0003267890732844 -0.0005914569365130, 0.0002673728781418 -0.0005833547866978)), ((0.0003159861287131 -0.0022821055312946, 0.0002619714058561 -0.0022902076811099, 0.0002619714058561 -0.0023118134139506, 0.0003213876009987 -0.0023280177135810, 0.0003159861287131 -0.0022821055312946)), ((0.0003132853925702 -0.0074431749636071, 0.0002619714058561 -0.0074566785466326, 0.0002835772949989 -0.0074674814130529, 0.0003132853925702 -0.0074431749636071)), ((0.0003105846564274 -0.0087179132012060, 0.0002916795034274 -0.0087044096181806, 0.0002754750865704 -0.0087260153510213, 0.0003105846564274 -0.0087179132012060)), ((0.0002916795034274 -0.0034245086552445, 0.0002997817118560 -0.0034515158212953, 0.0003078839202845 -0.0034299100884547, 0.0002916795034274 -0.0034245086552445)), ((0.0002430662528562 -0.0021902811667218, 0.0002808765588561 -0.0021983833165371, 0.0003078839202845 -0.0021713761504863, 0.0002700736142847 -0.0021551718508558, 0.0002430662528562 -0.0021902811667218)), ((0.0002943802395703 -0.0068139079946229, 0.0002592706697133 -0.0068220101444382, 0.0002565699335704 -0.0068490173104890, 0.0002970809757131 -0.0068355137274636, 0.0002943802395703 -0.0068139079946229)), ((0.0002916795034274 -0.0002538673608777, 0.0002484677251419 -0.0002241594782218, 0.0001998544745707 -0.0002376630612473, 0.0001701463769993 -0.0002889766767438, 0.0001782485854279 -0.0003267867092150, 0.0002214603637134 -0.0003294874258201, 0.0002241610998563 -0.0002835752435337, 0.0002916795034274 -0.0002538673608777)), ((0.0002619714058561 -0.0038377182958221, 0.0002673728781418 -0.0038647254618729, 0.0002835772949989 -0.0038485211622424, 0.0002619714058561 -0.0038377182958221)), ((0.0002727743504275 -0.0028249495689162, 0.0002268618359991 -0.0028195481357060, 0.0002214603637134 -0.0028573581681772, 0.0002673728781418 -0.0028519567349670, 0.0002727743504275 -0.0028249495689162)), ((0.0001917522661421 -0.0075458021946003, 0.0002322633082848 -0.0075728093606511, 0.0002727743504275 -0.0075647072108359, 0.0002727743504275 -0.0075485029112054, 0.0002133581552849 -0.0075295978949698, 0.0001917522661421 -0.0075458021946003)), ((0.0002646721419990 -0.0003537938752658, 0.0002268618359991 -0.0003808010413166, 0.0002646721419990 -0.0003753996081065, 0.0002646721419990 -0.0003537938752658)), ((0.0002592706697133 -0.0039052362109492, 0.0002187596275706 -0.0039214405105797, 0.0002376647805705 -0.0040078634419423, 0.0002592706697133 -0.0039997612921271, 0.0002403655167134 -0.0039592505430508, 0.0002619714058561 -0.0039403455268152, 0.0002592706697133 -0.0039052362109492)), ((0.0002403655167134 -0.0041969136042981, 0.0002484677251419 -0.0042266214869540, 0.0002619714058561 -0.0042050157541133, 0.0002403655167134 -0.0041969136042981)), ((0.0001512412239994 -0.0061144223939066, 0.0001782485854279 -0.0061306266935371, 0.0001917522661421 -0.0061063202440913, 0.0002268618359991 -0.0061549331429828, 0.0002592706697133 -0.0061522324263778, 0.0002403655167134 -0.0061009188108812, 0.0002106574191420 -0.0060847145112507, 0.0001512412239994 -0.0061144223939066)), ((0.0001944530022850 -0.0034866251371614, 0.0002133581552849 -0.0035190337364224, 0.0002376647805705 -0.0035055301533970, 0.0002241610998563 -0.0034785229873462, 0.0001944530022850 -0.0034866251371614)), ((0.0002349640444277 -0.0025926879408791, 0.0002187596275706 -0.0026196951069299, 0.0002376647805705 -0.0026169943903249, 0.0002349640444277 -0.0025926879408791)), ((0.0001998544745707 -0.0028870660508331, 0.0001728471131422 -0.0029221753666992, 0.0001917522661421 -0.0029518832493551, 0.0002268618359991 -0.0029221753666992, 0.0001998544745707 -0.0028870660508331)), ((0.0002268618359991 -0.0032273563430735, 0.0001836500577136 -0.0032273563430735, 0.0001998544745707 -0.0032543635091243, 0.0002268618359991 -0.0032273563430735)), ((0.0002241610998563 -0.0005185375881758, 0.0001917522661421 -0.0005563476206470, 0.0002079566829992 -0.0005806540700927, 0.0002241610998563 -0.0005185375881758)), ((0.0002133581552849 -0.0021740768670913, 0.0001863507938564 -0.0021794783003015, 0.0001890515299993 -0.0022172883327727, 0.0002187596275706 -0.0021956825999320, 0.0002133581552849 -0.0021740768670913)), ((0.0001944530022850 -0.0076565315754087, 0.0001593434324280 -0.0076754365916442, 0.0001674456408565 -0.0076997430410900, 0.0002133581552849 -0.0076835387414595, 0.0001944530022850 -0.0076565315754087)), ((0.0001782485854279 -0.0035595444854986, 0.0000945257649996 -0.0036027559511799, 0.0001566426962851 -0.0036135588176003, 0.0001080294457139 -0.0036486681334663, 0.0001323360709995 -0.0036918795991477, 0.0001755478492850 -0.0036945803157527, 0.0002052559468563 -0.0036540695666765, 0.0001782485854279 -0.0035595444854986)), ((0.0002025552107135 -0.0031220283954753, 0.0001620441685708 -0.0031328312618956, 0.0001593434324280 -0.0031706412943668, 0.0001917522661421 -0.0031625391445515, 0.0002025552107135 -0.0031220283954753)), ((0.0001971537384278 -0.0070272646064245, 0.0001674456408565 -0.0070542717724753, 0.0002025552107135 -0.0070542717724753, 0.0001971537384278 -0.0070272646064245)), ((0.0001539419601423 -0.0032813706751751, 0.0001242338625709 -0.0033137792744361, 0.0001323360709995 -0.0033542900235124, 0.0001593434324280 -0.0033542900235124, 0.0001971537384278 -0.0033110785578310, 0.0001539419601423 -0.0032813706751751)), ((0.0001917522661421 -0.0001593422796999, 0.0001755478492850 -0.0001296343970440, 0.0001269345987138 -0.0001404372634643, 0.0001296353348566 -0.0001593422796999, 0.0001917522661421 -0.0001593422796999)), ((0.0001809493215707 -0.0000216057328407, 0.0001026279734282 -0.0000216057328407, 0.0001350368071423 -0.0000891236479677, 0.0001863507938564 -0.0000486128988915, 0.0001809493215707 -0.0000216057328407)), ((0.0001755478492850 -0.0025656807748283, 0.0001350368071423 -0.0025791843578537, 0.0001350368071423 -0.0026223958235350, 0.0001863507938564 -0.0026061915239045, 0.0001755478492850 -0.0025656807748283)), ((0.0001809493215707 -0.0060658094950151, 0.0001404382794280 -0.0060739116448304, 0.0001701463769993 -0.0060874152278558, 0.0001809493215707 -0.0060658094950151)), ((0.0001377375432852 -0.0023415212966065, 0.0001107301818567 -0.0023523241630268, 0.0001188323902853 -0.0023820320456827, 0.0000972265011425 -0.0024009370619183, 0.0001485404878566 -0.0024387470943894, 0.0001701463769993 -0.0024144406449437, 0.0001269345987138 -0.0023901341954979, 0.0001593434324280 -0.0023577255962370, 0.0001377375432852 -0.0023415212966065)), ((0.0001242338625709 -0.0042239207703489, 0.0001188323902853 -0.0042563293696099, 0.0001593434324280 -0.0042644315194251, 0.0001593434324280 -0.0042293222035591, 0.0001242338625709 -0.0042239207703489)), ((0.0001593434324280 -0.0067058793304196, 0.0000891242927139 -0.0067085800470247, 0.0000891242927139 -0.0067733972455467, 0.0001431390155709 -0.0067787986787569, 0.0001593434324280 -0.0067058793304196)), ((0.0001269345987138 -0.0028546574515721, 0.0001296353348566 -0.0029248760833043, 0.0001566426962851 -0.0029194746500941, 0.0001485404878566 -0.0028600588847823, 0.0001269345987138 -0.0028546574515721)), ((0.0001485404878566 -0.0039133383607644, 0.0000945257649996 -0.0039484476766305, 0.0001404382794280 -0.0039565498264457, 0.0001485404878566 -0.0039133383607644)), ((0.0001242338625709 -0.0026386001231655, 0.0001188323902853 -0.0026602058560062, 0.0001431390155709 -0.0026656072892163, 0.0001242338625709 -0.0026386001231655)), ((0.0001404382794280 -0.0062710639570014, 0.0001161316541424 -0.0062521589407658, 0.0000567154589998 -0.0062818668234217, 0.0000594161951426 -0.0063358811555234, 0.0001080294457139 -0.0063304797223132, 0.0001404382794280 -0.0062710639570014)), ((0.0001026279734282 -0.0030788169297940, 0.0000918250288568 -0.0031193276788702, 0.0001134309179996 -0.0031301305452905, 0.0001350368071423 -0.0031004226626346, 0.0001026279734282 -0.0030788169297940)), ((0.0000486132505712 -0.0002241594782218, 0.0000972265011425 -0.0002619695106930, 0.0001323360709995 -0.0002457652110625, 0.0001161316541424 -0.0002133566118015, 0.0000486132505712 -0.0002241594782218)), ((0.0001269345987138 -0.0027709352368146, 0.0000972265011425 -0.0028033438360756, 0.0001269345987138 -0.0027979424028654, 0.0001269345987138 -0.0027709352368146)), ((0.0000243066252856 -0.0037404924980392, 0.0000783213481426 -0.0037620982308798, 0.0000999272372853 -0.0037458939312493, 0.0000567154589998 -0.0037188867651985, 0.0000243066252856 -0.0037404924980392)), ((0.0000702191397140 -0.0056390962714121, 0.0000756206119997 -0.0056553005710426, 0.0000972265011425 -0.0056471984212273, 0.0000702191397140 -0.0056390962714121)), ((0.0000756206119997 -0.0028519567349670, 0.0000540147228569 -0.0028573581681772, 0.0000432117782855 -0.0029005696338585, 0.0000027007361428 -0.0029140732168839, 0.0000108029445714 -0.0029977954316415, 0.0000675184035712 -0.0029761896988008, 0.0000945257649996 -0.0028978689172534, 0.0000756206119997 -0.0028519567349670)), ((0.0000918250288568 -0.0031625391445515, 0.0000459125144284 -0.0032030498936277, 0.0000486132505712 -0.0032273563430735, 0.0000918250288568 -0.0031625391445515)), ((0.0000891242927139 -0.0038188132795865, 0.0000486132505712 -0.0038242147127967, 0.0000378103059999 -0.0038485211622424, 0.0000675184035712 -0.0038485211622424, 0.0000891242927139 -0.0038188132795865)), ((0.0000864235565711 -0.0043643580338132, 0.0000594161951426 -0.0043697594670234, 0.0000513139867141 -0.0043967666330742, 0.0000864235565711 -0.0043643580338132)), ((0.0000108029445714 -0.0005212383047809, 0.0000324088337142 -0.0005617490538571, 0.0000756206119997 -0.0005590483372521, 0.0000675184035712 -0.0005239390213860, 0.0000108029445714 -0.0005212383047809)), ((0.0000702191397140 -0.0022253904825879, 0.0000405110421427 -0.0022334926324032, 0.0000594161951426 -0.0022713026648743, 0.0000702191397140 -0.0022253904825879)), ((0.0000675184035712 -0.0070704760721058, 0.0000297080975713 -0.0070596732056854, 0.0000108029445714 -0.0071136875377871, 0.0000486132505712 -0.0071055853879718, 0.0000675184035712 -0.0070704760721058)), ((0.0000540147228569 -0.0031139262456600, 0.0000270073614285 -0.0031085248124499, 0.0000108029445714 -0.0031382326951058, 0.0000432117782855 -0.0031355319785007, 0.0000540147228569 -0.0031139262456600)), ((0.0000297080975713 -0.0002835752435337, 0.0000540147228569 -0.0003132831261896, 0.0000540147228569 -0.0002862759601387, 0.0000297080975713 -0.0002835752435337)), ((0.0000081022084285 -0.0049531142537212, 0.0000108029445714 -0.0050017271526126, 0.0000540147228569 -0.0049990264360076, 0.0000540147228569 -0.0049693185533517, 0.0000081022084285 -0.0049531142537212)), ((0.0000270073614285 -0.0045182988803029, 0.0000027007361428 -0.0045264010301181, 0.0000027007361428 -0.0045750139290096, 0.0000405110421427 -0.0045696124957995, 0.0000270073614285 -0.0045182988803029)), ((0.0000027007361428 -0.0033434871570920, 0.0000108029445714 -0.0033839979061683, 0.0000405110421427 -0.0033785964729581, 0.0000270073614285 -0.0033434871570920, 0.0000027007361428 -0.0033434871570920)), ((0.0000027007361428 -0.0030680140633736, 0.0000351095698570 -0.0030869190796092, 0.0000324088337142 -0.0030680140633736, 0.0000027007361428 -0.0030680140633736)), ((0.0000027007361428 -0.0035433401858681, 0.0000027007361428 -0.0035730480685240, 0.0000243066252856 -0.0035703473519190, 0.0000324088337142 -0.0035487416190783, 0.0000027007361428 -0.0035433401858681)))"])
        writer.writerow(['6010_1_2','6',"MULTIPOLYGON (((0.0091284881628216 -0.0087665261000975, 0.0090339623978220 -0.0088016354159636, 0.0091257874266788 -0.0088016354159636, 0.0091284881628216 -0.0087665261000975)), ((0.0089124292713939 -0.0088016354159636, 0.0090285609255363 -0.0088016354159636, 0.0090204587171078 -0.0087071103347857, 0.0090906778568218 -0.0086963074683654, 0.0089097285352511 -0.0086963074683654, 0.0089124292713939 -0.0088016354159636), (0.0089745462026794 -0.0087611246668874, 0.0089907506195365 -0.0087692268167026, 0.0089799476749651 -0.0087854311163331, 0.0089745462026794 -0.0087611246668874)), ((0.0091419918435359 -0.0084802501399588, 0.0091284881628216 -0.0085855780875570, 0.0091500940519644 -0.0085558702049011, 0.0091419918435359 -0.0084802501399588)), ((0.0057957797625489 -0.0005590483372521, 0.0058470937492630 -0.0005914569365130, 0.0058281885962630 -0.0005563476206470, 0.0057957797625489 -0.0005590483372521)), ((0.0045912514428393 -0.0003781003247115, 0.0045912514428393 -0.0004834282723098, 0.0046371639572677 -0.0004834282723098, 0.0046128573319821 -0.0004186110737878, 0.0046965801524103 -0.0004645232560742, 0.0046857772078389 -0.0003916039077370, 0.0045912514428393 -0.0003781003247115)))"])
        writer.writerow(['6010_1_2','7','GEOMETRYCOLLECTION EMPTY'])
        writer.writerow(['6010_1_2','8','GEOMETRYCOLLECTION EMPTY'])
        writer.writerow(['6010_1_2','9','GEOMETRYCOLLECTION EMPTY'])
        writer.writerow(['6010_1_2','10','GEOMETRYCOLLECTION EMPTY'])


In [ ]:
train_pred_fn = os.path.join('..','input', 'train_wkt_v4.csv')

test_size = (3000,3000)
num_classes = 10


with open('fake_pred.csv','r') as fp:
    pred_reader = csv.reader(fp)
    header = next(pred_reader)
    with open(train_pred_fn,'r') as tp:
        train_reader = csv.reader(tp)
        header = next(train_reader)
        
        old_imageid = ''
        
        # here only expect 1 imageid from pred_reader
        for row_p in pred_reader:
#            print('searching for {}'.format(row_p[:2]))
            found = False
            while not found:
                row_t = next(train_reader)
                if (row_p[:2] == row_t[:2]):
                    found = True
            imageid = row_p[0]
            clsid = int(row_p[1])
            if old_imageid != imageid:
                old_imageid = imageid
                pred = np.empty( (num_classes,) + test_size, dtype=np.bool)
                target = np.empty( (num_classes,) + test_size, dtype=np.bool)
                x_scaler, y_scaler = get_scalers_for_image(imageid, test_size)

            pred_mp = shapely.wkt.loads(row_p[2])
            target_mp = shapely.wkt.loads(row_t[2])
            
            pred_mp_scaled = scale_wkt_to_mask(pred_mp, x_scaler, y_scaler)
            target_mp_scaled = scale_wkt_to_mask(target_mp, x_scaler, y_scaler)

            pred[(clsid - 1)] = mask_for_polygons(pred_mp_scaled, test_size)
            target[(clsid - 1)] = mask_for_polygons(pred_mp_scaled, test_size)
            
            if clsid == 10:
                # compute jaccard
                jaccard_by_class = compute_jaccard_from_masks(pred, target)
                for cls, score in enumerate(jaccard_by_class):
                    print('score for {} class {} is {}'.format(imageid, cls+1, score))
                
            


